# Chapter 9: Classes and Objects

## Introduction

SystemVerilog introduces object-oriented programming (OOP) concepts to hardware description and verification. Classes provide a powerful way to create reusable, modular code structures that can model complex data types and behaviors. This chapter covers the fundamental concepts of classes and objects in SystemVerilog.

## Class Declarations

A class in SystemVerilog is a user-defined data type that encapsulates data (properties) and functions (methods) that operate on that data. Classes serve as templates for creating objects.

### Basic Class Syntax

```systemverilog
class ClassName;
    // Properties (data members)
    // Methods (functions and tasks)
endclass
```

### Simple Class Example

```systemverilog
class Packet;
    // Properties
    bit [7:0] header;
    bit [31:0] payload;
    bit [7:0] checksum;
    
    // Method to display packet contents
    function void display();
        $display("Header: %h, Payload: %h, Checksum: %h", 
                 header, payload, checksum);
    endfunction
endclass
```

### Class with Constructor

```systemverilog
class Transaction;
    rand bit [31:0] addr;
    rand bit [31:0] data;
    bit [1:0] cmd;
    
    // Constructor
    function new(bit [1:0] command = 0);
        cmd = command;
        // Randomize other fields
        assert(randomize());
    endfunction
    
    // Method to check transaction validity
    function bit is_valid();
        return (addr != 0 && cmd != 2'b11);
    endfunction
endclass
```

### **Example 1: Basic Class Template**
Simple class structure showing the fundamental syntax with properties and a basic method.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_1__basic_class_template/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// basic_class_example_testbench.sv
import counter_pkg::*;                      // Import all items from package

module basic_class_test_bench;              // Testbench module
  simple_counter_class my_counter;          // Class handle (object reference)
  simple_counter_class second_counter;      // Another class handle
  
  initial begin
    // Dump waves for simulation
    $dumpfile("basic_class_test_bench.vcd");
    $dumpvars(0, basic_class_test_bench);
    
    $display("=== Basic Class Template Example (Package Version) ===");
    $display();
    
    // Create first counter object
    my_counter = new("primary_counter");    // Instantiate class object
    
    // Test basic operations on first counter
    my_counter.increment();                 // Call increment method
    my_counter.increment();                 // Increment again
    my_counter.increment();                 // One more time
    
    $display("Current count: %0d", my_counter.get_count());
    $display();
    
    // Create second counter object with different name
    second_counter = new("backup_counter"); // Another object instance
    
    // Test operations on second counter
    second_counter.increment();             // This counter is independent
    $display("Second counter: %0d", second_counter.get_count());
    $display("First counter still: %0d", my_counter.get_count());
    $display();
    
    // Test reset functionality
    my_counter.reset();                     // Reset first counter
    $display("After reset: %0d", my_counter.get_count());
    
    $display();
    $display("=== Package-Based Class Example Complete ===");
    
    #1;                                     // Wait for a time unit
  end

endmodule
```

```systemverilog
// counter_package.sv
package counter_pkg;                        // SystemVerilog package definition

  class simple_counter_class;               // Basic class definition
    int count_value;                        // Class property (data member)
    string counter_name;                    // Another class property
    
    // Constructor method - called when object is created
    function new(string name = "default_counter");
      counter_name = name;                  // Initialize counter name
      count_value = 0;                      // Initialize count to zero
      $display("Created counter: %s", counter_name);
    endfunction
    
    // Method to increment the counter
    function void increment();
      count_value++;                        // Increment the count
      $display("%s: count = %0d", counter_name, count_value);
    endfunction
    
    // Method to get current count value
    function int get_count();
      return count_value;                   // Return current count
    endfunction
    
    // Method to reset the counter
    function void reset();
      count_value = 0;                      // Reset count to zero
      $display("%s: reset to 0", counter_name);
    endfunction

  endclass

endpackage : counter_pkg                    // End package with explicit name
```

Verilator Simulation Output:
=== Basic Class Template Example (Package Version) ===

Created counter: primary_counter
primary_counter: count = 1
primary_counter: count = 2
primary_counter: count = 3
Current count: 3

Created counter: backup_counter
backup_counter: count = 1
Second counter: 1
First counter still: 3

primary_counter: reset to 0
After reset: 0

Process finished with return code: 0
Removing Chapter_9_examples/example_1__basic_class_template/obj_dir directory...
Chapter_9_examples/example_1__basic_class_template/obj_dir removed successfully.


0

### **Example 2: Simple Packet Class**
Basic class representing a network packet with header, payload, and display functionality.

In [18]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_2__simple_packet_class/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_packet_class.sv
class NetworkPacket;
  // Class properties
  rand bit [15:0] header_length;    // Header length in bytes
  rand bit [31:0] packet_id;        // Unique packet identifier
  rand bit [7:0]  packet_type;      // Type of packet (TCP, UDP, etc.)
  rand bit [15:0] payload_size;     // Payload size in bytes
  rand bit [7:0]  payload_data[];   // Dynamic array for payload
  
  // Constructor
  function new(bit [15:0] hdr_len = 20, bit [15:0] payload_sz = 64);
    // Set fixed values first
    this.header_length = hdr_len;
    this.payload_size = payload_sz;
    
    // Create payload array with correct size
    this.payload_data = new[int'(payload_sz)];
    
    // Set random values for ID and type
    this.packet_id = $urandom();
    this.packet_type = 8'($urandom_range(0, 255));
    
    // Fill payload with pattern
    foreach(payload_data[i]) begin
      payload_data[i] = 8'(i % 256);
    end
  endfunction
  
  // Method to display packet information
  function void display_packet_info();
    $display("=== Network Packet Information ===");
    $display("Header Length: %0d bytes", header_length);
    $display("Packet ID:     0x%08h", packet_id);
    $display("Packet Type:   0x%02h", packet_type);
    $display("Payload Size:  %0d bytes", payload_size);
    $display("Total Size:    %0d bytes", header_length + payload_size);
  endfunction
  
  // Method to display first few bytes of payload
  function void display_payload_sample(int num_bytes = 8);
    $display("--- Payload Sample (first %0d bytes) ---", num_bytes);
    $write("Payload: ");
    for(int i = 0; i < num_bytes && i < payload_data.size(); i++) begin
      $write("0x%02h ", payload_data[i]);
    end
    $display();
  endfunction
  
  // Method to calculate total packet size
  function int get_total_size();
    return int'(header_length) + int'(payload_size);
  endfunction
  
  // Method to check if packet is valid
  function bit is_valid_packet();
    return (header_length > 0) && (payload_size > 0) && 
           (int'(payload_data.size()) == int'(payload_size));
  endfunction
  
endclass : NetworkPacket

// Design module that demonstrates packet usage
module packet_processor;
  NetworkPacket pkt;
  
  initial begin
    $display();
    $display("=== Packet Processor Module ===");
    
    // Create a packet instance
    pkt = new(24, 128);  // 24-byte header, 128-byte payload
    
    // Display packet information
    pkt.display_packet_info();
    pkt.display_payload_sample(12);
    
    // Validate packet
    if(pkt.is_valid_packet()) begin
      $display("Packet validation: PASSED");
    end else begin
      $display("Packet validation: FAILED");
    end
    
    $display("Total packet size: %0d bytes", pkt.get_total_size());
    $display();
  end
  
endmodule : packet_processor
```

```systemverilog
// simple_packet_class_testbench.sv
module packet_testbench;
  // Instantiate the packet processor design
  packet_processor PACKET_PROC_INSTANCE();
  
  // Local packet instances for testing
  NetworkPacket test_packet_1;
  NetworkPacket test_packet_2;
  NetworkPacket test_packet_3;
  
  initial begin
    // Setup for waveform dumping
    $dumpfile("packet_testbench.vcd");
    $dumpvars(0, packet_testbench);
    
    $display();
    $display("Testbench starting...");
    $display();
    
    // Wait for design module to complete
    #1;
    
    // Test 1: Create small packet
    $display("=== Test 1: Small Packet ===");
    test_packet_1 = new(16, 32);  // Small packet: 16-byte header, 32-byte payload
    test_packet_1.display_packet_info();
    test_packet_1.display_payload_sample(6);
    $display("Is valid: %s", test_packet_1.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 2: Create large packet
    $display("=== Test 2: Large Packet ===");
    test_packet_2 = new(40, 1024); // Large packet: 40-byte header, 1KB payload
    test_packet_2.display_packet_info();
    test_packet_2.display_payload_sample(10);
    $display("Is valid: %s", test_packet_2.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 3: Create standard packet and randomize
    $display("=== Test 3: Randomized Packet ===");
    test_packet_3 = new();  // Use default constructor
    // Re-randomize only packet_id and packet_type
    test_packet_3.packet_id = $urandom();
    test_packet_3.packet_type = 8'($urandom_range(0, 255));
    test_packet_3.display_packet_info();
    test_packet_3.display_payload_sample(8);
    $display("Is valid: %s", test_packet_3.is_valid_packet() ? "YES" : "NO");
    $display();
    
    // Test 4: Compare packet sizes
    $display("=== Test 4: Packet Size Comparison ===");
    $display("Packet 1 total size: %0d bytes", test_packet_1.get_total_size());
    $display("Packet 2 total size: %0d bytes", test_packet_2.get_total_size());
    $display("Packet 3 total size: %0d bytes", test_packet_3.get_total_size());
    
    if(test_packet_2.get_total_size() > test_packet_1.get_total_size()) begin
      $display("Packet 2 is larger than Packet 1 (as expected)");
    end
    $display();
    
    // Final summary
    $display("=== Testbench Summary ===");
    $display("Created and tested 3 different packet instances");
    $display("Demonstrated class instantiation and method calls");
    $display("Verified packet validation and size calculation");
    $display("Showed dynamic array usage in payload");
    $display();
    
    $display("Hello from packet class testbench!");
    $display();
    
    // Finish simulation
    $finish;
  end
  
endmodule : packet_testbench
```

Verilator Simulation Output:

=== Packet Processor Module ===
=== Network Packet Information ===
Header Length: 24 bytes
Packet ID:     0x797673c4
Packet Type:   0x9c
Payload Size:  128 bytes
Total Size:    152 bytes
--- Payload Sample (first 12 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05 0x06 0x07 0x08 0x09 0x0a 0x0b
Packet validation: PASSED
Total packet size: 152 bytes


Testbench starting...

=== Test 1: Small Packet ===
=== Network Packet Information ===
Header Length: 16 bytes
Packet ID:     0x9efdd502
Packet Type:   0xe4
Payload Size:  32 bytes
Total Size:    48 bytes
--- Payload Sample (first 6 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05
Is valid: YES

=== Test 2: Large Packet ===
=== Network Packet Information ===
Header Length: 40 bytes
Packet ID:     0x92178378
Packet Type:   0xbc
Payload Size:  1024 bytes
Total Size:    1064 bytes
--- Payload Sample (first 10 bytes) ---
Payload: 0x00 0x01 0x02 0x03 0x04 0x05 0x06 0x07 0x08 0x09
Is valid: YES

=== Test 3: Randomi

0

### **Example 3: Transaction with Constructor**
Class demonstrating constructor usage with default parameters and basic validation.

In [28]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_3__transaction_constructor/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// packet_transaction.sv
class packet_transaction;
  // Transaction properties
  rand bit [7:0]  src_addr;
  rand bit [7:0]  dst_addr; 
  rand bit [15:0] payload_size;
  rand bit [1:0]  pkt_priority;
  bit             valid;
  
  // Constructor with default parameters and validation
  function new(bit [7:0] src = 8'h00, 
               bit [7:0] dst = 8'hFF,
               bit [15:0] size = 16'd64,
               bit [1:0] prio = 2'b01);
    
    $display("Creating new packet transaction...");
    
    // Assign parameters to class properties
    src_addr = src;
    dst_addr = dst;
    payload_size = size;
    pkt_priority = prio;
    
    // Basic validation
    if (validate_transaction()) begin
      valid = 1'b1;
      $display("Transaction validation: PASSED");
    end else begin
      valid = 1'b0;
      $display("Transaction validation: FAILED");
    end
    
    $display("Constructor completed");
    $display();
  endfunction
  
  // Validation function
  function bit validate_transaction();
    bit validation_result = 1'b1;
    
    // Check if source and destination are different
    if (src_addr == dst_addr) begin
      $display("ERROR: Source and destination addresses are the same!");
      validation_result = 1'b0;
    end
    
    // Check payload size limits (minimum 1, maximum 1024)
    if (payload_size == 0 || payload_size > 1024) begin
      $display("ERROR: Invalid payload size: %0d", payload_size);
      validation_result = 1'b0;
    end
    
    return validation_result;
  endfunction
  
  // Display transaction details
  function void display_transaction();
    $display("=== Packet Transaction Details ===");
    $display("Source Address:  0x%02h", src_addr);
    $display("Dest Address:    0x%02h", dst_addr);
    $display("Payload Size:    %0d bytes", payload_size);
    $display("Priority:        %0d", pkt_priority);
    $display("Valid:           %0s", valid ? "YES" : "NO");
    $display("===================================");
    $display();
  endfunction
  
endclass
```

```systemverilog
// packet_transaction_testbench.sv
module packet_constructor_testbench;
  
  // Transaction handles
  packet_transaction default_packet;
  packet_transaction custom_packet;
  packet_transaction invalid_packet;
  packet_transaction partial_packet;
  
  initial begin
    // Dump waves for visualization
    $dumpfile("packet_constructor_testbench.vcd");
    $dumpvars(0, packet_constructor_testbench);
    
    $display();
    
    // Test 1: Create packet with default constructor parameters
    $display("Test 1: Creating packet with default parameters");
    $display("--------------------------------------------------------");
    default_packet = new();  // Uses all default values
    default_packet.display_transaction();
    
    // Test 2: Create packet with custom parameters
    $display("Test 2: Creating packet with custom parameters");
    $display("--------------------------------------------------------");
    custom_packet = new(8'hA0, 8'hB5, 16'd256, 2'b11);
    custom_packet.display_transaction();
    
    // Test 3: Create invalid packet (same src/dst) to test validation
    $display("Test 3: Creating invalid packet (same source/destination)");
    $display("--------------------------------------------------------");
    invalid_packet = new(8'hCC, 8'hCC, 16'd128, 2'b10);  // Same src/dst
    invalid_packet.display_transaction();
    
    // Test 4: Demonstrate partial parameter usage
    $display("Test 4: Creating packet with partial custom parameters");
    $display("--------------------------------------------------------");
    partial_packet = new(8'h11, 8'hFF, 16'd512, 2'b01);  // Custom src & size
    partial_packet.display_transaction();
    
    $display("=== Constructor Demo Completed ===");
    $display();
    
    #10;  // Small delay before ending
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

Test 1: Creating packet with default parameters
--------------------------------------------------------
Creating new packet transaction...
Transaction validation: PASSED
Constructor completed

=== Packet Transaction Details ===
Source Address:  0x00
Dest Address:    0xff
Payload Size:    64 bytes
Priority:        1
Valid:           YES

Test 2: Creating packet with custom parameters
--------------------------------------------------------
Creating new packet transaction...
Transaction validation: PASSED
Constructor completed

=== Packet Transaction Details ===
Source Address:  0xa0
Dest Address:    0xb5
Payload Size:    256 bytes
Priority:        3
Valid:           YES

Test 3: Creating invalid packet (same source/destination)
--------------------------------------------------------
Creating new packet transaction...
ERROR: Source and destination addresses are the same!
Transaction validation: FAILED
Constructor completed

=== Packet Transaction Details =

0

## Properties and Methods

Properties are the data members of a class, while methods are the functions and tasks that operate on the class data.

### Property Types

```systemverilog
class DataPacket;
    // Basic properties
    bit [7:0] id;
    int length;
    real timestamp;
    string source;
    
    // Array properties
    bit [7:0] data[];
    int status_flags[4];
    
    // Random properties
    rand bit [15:0] sequence_num;
    randc bit [3:0] priority;
    
    // Constraints on random properties
    constraint valid_priority {
        priority inside {[1:8]};
    }
    
    constraint data_size {
        length > 0;
        length < 1024;
        data.size() == length;
    }
endclass
```

### Method Types

```systemverilog
class NetworkPacket;
    bit [47:0] src_mac;
    bit [47:0] dst_mac;
    bit [15:0] ethertype;
    bit [7:0] payload[];
    
    // Constructor
    function new(bit [47:0] src = 0, bit [47:0] dst = 0);
        src_mac = src;
        dst_mac = dst;
        ethertype = 16'h0800; // IP
    endfunction
    
    // Function method (returns a value)
    function int get_payload_size();
        return payload.size();
    endfunction
    
    // Task method (can consume time)
    task send_packet();
        #10ns; // Simulate transmission delay
        $display("Packet sent from %h to %h", src_mac, dst_mac);
    endtask
    
    // Virtual method (can be overridden)
    virtual function void print_header();
        $display("SRC: %h, DST: %h, Type: %h", 
                 src_mac, dst_mac, ethertype);
    endfunction
    
    // Static method (belongs to class, not instance)
    static function bit [15:0] calculate_checksum(bit [7:0] data[]);
        bit [15:0] sum = 0;
        foreach(data[i]) sum += data[i];
        return ~sum;
    endfunction
endclass
```

### **Example 4: Data Packet Properties**
Class showcasing different types of properties: basic types, arrays, random variables with constraints.

In [35]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_4__data_packet_properties/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_packet_properties.sv
class DataPacket;
  // Basic properties
  bit [7:0]   packet_id;               // Packet identifier
  bit [15:0]  packet_length;           // Packet length
  string      packet_type;             // Packet type description
  
  // Array properties
  bit [7:0]   payload_data[];          // Dynamic array for payload
  bit [31:0]  header_fields[4];        // Fixed array for header fields
  
  // Additional properties
  bit [3:0]   priority_level;          // Packet priority (0-15)
  bit [7:0]   destination_addr;        // Destination address
  bit         valid_packet;            // Packet validity flag
  
  // Constructor
  function new();
    packet_type = "DATA";              // Default packet type
    valid_packet = 1'b1;               // Default to valid
    priority_level = 4'd3;             // Default priority
    destination_addr = 8'h42;          // Default destination
    packet_id = 8'h01;                 // Default ID
    packet_length = 16'd128;           // Default length
    
    // Initialize header fields
    foreach(header_fields[i]) begin
      header_fields[i] = 32'hDEADBEEF + i;
    end
    
    // Initialize payload with default size
    set_payload_size(108);             // 128 - 20 byte header
  endfunction
  
  // Method to set payload size and initialize with pattern
  function void set_payload_size(int size);
    payload_data = new[size];
    foreach(payload_data[i]) begin
      payload_data[i] = 8'(8'hA0 + (i % 16));
    end
  endfunction
  
  // Method to create a control packet
  function void make_control_packet();
    packet_type = "CONTROL";
    packet_length = 16'd64;
    priority_level = 4'd6;             // High priority
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'h10;          // Control address
    set_payload_size(44);              // 64 - 20 byte header
  endfunction
  
  // Method to create a broadcast packet
  function void make_broadcast_packet();
    packet_type = "BROADCAST";
    packet_length = 16'd256;
    priority_level = 4'd7;             // Maximum priority
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'hFE;          // Near-broadcast address
    set_payload_size(236);             // 256 - 20 byte header
  endfunction
  
  // Method to create a data packet with random properties
  function void make_data_packet();
    packet_type = "DATA";
    packet_length = 16'(64 + ($urandom() % 1455)); // 64-1518 range
    priority_level = 4'(1 + ($urandom() % 7));     // 1-7 range
    packet_id = 8'($urandom() & 8'hFF);
    destination_addr = 8'(1 + ($urandom() % 254)); // 1-254 range
    set_payload_size(int'(packet_length) - 20);
  endfunction
  
  // Method to display packet information
  function void display_packet();
    $display("=== Data Packet Information ===");
    $display("Packet ID: 0x%02h", packet_id);
    $display("Packet Type: %s", packet_type);
    $display("Packet Length: %0d bytes", packet_length);
    $display("Priority Level: %0d", priority_level);
    $display("Destination Address: 0x%02h", destination_addr);
    $display("Valid Packet: %s", valid_packet ? "YES" : "NO");
    $display("Payload Size: %0d bytes", payload_data.size());
    
    // Display header fields
    $display("Header Fields:");
    foreach(header_fields[i]) begin
      $display("  Field[%0d]: 0x%08h", i, header_fields[i]);
    end
    
    // Display first few payload bytes
    if (payload_data.size() > 0) begin
      $display("Payload Preview (first 8 bytes):");
      for (int i = 0; i < 8 && i < payload_data.size(); i++) begin
        $display("  payload_data[%0d]: 0x%02h", i, payload_data[i]);
      end
    end
    $display("===============================");
  endfunction
  
endclass
```

```systemverilog
// data_packet_properties_testbench.sv
module packet_testbench;
  
  // Create packet instances
  DataPacket data_pkt;
  DataPacket control_pkt;
  DataPacket broadcast_pkt;
  
  initial begin
    // Dump waves for simulation
    $dumpfile("packet_testbench.vcd");
    $dumpvars(0, packet_testbench);
    
    $display();
    
    // Test 1: Create and configure a standard data packet
    $display("--- Test 1: Standard Data Packet ---");
    data_pkt = new();
    data_pkt.make_data_packet();
    data_pkt.display_packet();
    $display();
    
    // Test 2: Create a control packet
    $display("--- Test 2: Control Packet ---");
    control_pkt = new();
    control_pkt.make_control_packet();
    control_pkt.display_packet();
    $display();
    
    // Test 3: Create a broadcast packet
    $display("--- Test 3: High Priority Broadcast Packet ---");
    broadcast_pkt = new();
    broadcast_pkt.make_broadcast_packet();
    broadcast_pkt.display_packet();
    $display();
    
    // Test 4: Demonstrate array property manipulation
    $display("--- Test 4: Array Property Manipulation ---");
    data_pkt.packet_type = "MODIFIED";
    
    // Manually modify some array elements
    data_pkt.header_fields[0] = 32'hCAFEBABE;
    data_pkt.header_fields[1] = 32'h12345678;
    
    // Modify payload with custom pattern
    data_pkt.set_payload_size(16);
    for (int i = 0; i < data_pkt.payload_data.size(); i++) begin
      data_pkt.payload_data[i] = 8'(8'hF0 + i);
    end
    data_pkt.packet_length = 16'd36; // 16 + 20 header
    
    data_pkt.display_packet();
    $display();
    
    // Test 5: Create multiple different packet types
    $display("--- Test 5: Multiple Packet Variations ---");
    for (int test_num = 1; test_num <= 3; test_num++) begin
      $display("Creating packet variation #%0d:", test_num);
      data_pkt.make_data_packet();
      $display(
        "  ID: 0x%02h, Length: %0d, Priority: %0d, Dest: 0x%02h",
        data_pkt.packet_id, data_pkt.packet_length, 
        data_pkt.priority_level, data_pkt.destination_addr);
    end
    $display();
    
    // Test 6: Demonstrate different packet sizes
    $display("--- Test 6: Different Packet Sizes ---");
    
    // Small packet
    data_pkt.packet_type = "SMALL";
    data_pkt.packet_length = 16'd64;
    data_pkt.set_payload_size(44);
    $display(
        "Small packet - Length: %0d, Payload: %0d bytes", 
        data_pkt.packet_length, data_pkt.payload_data.size());
    
    // Large packet
    data_pkt.packet_type = "LARGE";
    data_pkt.packet_length = 16'd1518;
    data_pkt.set_payload_size(1498);
    $display(
        "Large packet - Length: %0d, Payload: %0d bytes", 
        data_pkt.packet_length, data_pkt.payload_data.size());
    $display();
    
    $display("========================================");
    $display("Simulation completed successfully!");
    $display("All packet properties demonstrated:");
    $display("- Basic types (ID, length, type)");
    $display("- Arrays (fixed header, dynamic payload)");
    $display("- Property manipulation methods");
    $display("- Different packet configurations");
    $display("========================================");
    
    #10;  // Wait before finishing
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

--- Test 1: Standard Data Packet ---
=== Data Packet Information ===
Packet ID: 0x02
Packet Type: DATA
Packet Length: 1469 bytes
Priority Level: 5
Destination Address: 0xfb
Valid Packet: YES
Payload Size: 1449 bytes
Header Fields:
  Field[0]: 0xdeadbeef
  Field[1]: 0xdeadbef0
  Field[2]: 0xdeadbef1
  Field[3]: 0xdeadbef2
Payload Preview (first 8 bytes):
  payload_data[0]: 0xa0
  payload_data[1]: 0xa1
  payload_data[2]: 0xa2
  payload_data[3]: 0xa3
  payload_data[4]: 0xa4
  payload_data[5]: 0xa5
  payload_data[6]: 0xa6
  payload_data[7]: 0xa7

--- Test 2: Control Packet ---
=== Data Packet Information ===
Packet ID: 0x78
Packet Type: CONTROL
Packet Length: 64 bytes
Priority Level: 6
Destination Address: 0x10
Valid Packet: YES
Payload Size: 44 bytes
Header Fields:
  Field[0]: 0xdeadbeef
  Field[1]: 0xdeadbef0
  Field[2]: 0xdeadbef1
  Field[3]: 0xdeadbef2
Payload Preview (first 8 bytes):
  payload_data[0]: 0xa0
  payload_data[1]: 0xa1
  payload_data[2]: 0xa2


0

### **Example 5: Network Packet Methods**
Demonstrates different method types: constructor, function, task, virtual, and static methods.

In [5]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_5__network_packet_methods/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// network_packet_methods.sv
// Chapter 9 Example 5: Network Packet Methods
// Demonstrates constructor, function, task, virtual, and static methods

class network_packet_base;
  // Packet properties
  rand bit [7:0]  packet_id;
  rand bit [15:0] packet_size;
  rand bit [31:0] source_addr;
  rand bit [31:0] dest_addr;
  static int      packet_count = 0;  // Static counter for all packets
  
  // Constructor method
  function new(bit [31:0] src = 32'h0, bit [31:0] dst = 32'hFFFFFFFF);
    source_addr = src;
    dest_addr = dst;
    packet_count++;  // Increment static counter
    $display("Packet created: ID=%0d, Count=%0d", packet_id, packet_count);
  endfunction
  
  // Function method - returns checksum (pure function)
  function bit [15:0] calculate_checksum();
    return {8'h0, packet_id} + {8'h0, packet_size[7:0]} + {8'h0, packet_size[15:8]};
  endfunction
  
  // Task method - displays packet info (can consume time)
  virtual task display_packet_info();
    #1ns;  // Simulate time delay
    $display("=== Packet Info ===");
    $display("ID: 0x%02h, Size: %0d bytes", packet_id, packet_size);
    $display("From: 0x%08h To: 0x%08h", source_addr, dest_addr);
    $display("Checksum: 0x%04h", calculate_checksum());
  endtask
  
  // Virtual function - can be overridden in derived classes
  virtual function string get_packet_type();
    return "BASE";
  endfunction
  
  // Static method - operates on class level, not instance level
  static function int get_total_packet_count();
    return packet_count;
  endfunction
  
endclass

// Derived class demonstrating virtual method override
class tcp_packet extends network_packet_base;
  rand bit [15:0] tcp_port_src;
  rand bit [15:0] tcp_port_dst;
  
  // Constructor calls parent constructor
  function new(bit [31:0] src = 32'h0, bit [31:0] dst = 32'hFFFFFFFF);
    super.new(src, dst);
    tcp_port_src = 80;   // Default HTTP port
    tcp_port_dst = 8080; // Default alt HTTP port
  endfunction
  
  // Override virtual function
  virtual function string get_packet_type();
    return "TCP";
  endfunction
  
  // Override virtual task
  virtual task display_packet_info();
    super.display_packet_info();  // Call parent method
    $display("TCP Ports: %0d -> %0d", tcp_port_src, tcp_port_dst);
  endtask
  
endclass

module network_packet_methods_demo;
  
  initial begin
    network_packet_base base_pkt;
    tcp_packet         tcp_pkt;
    
    $display("=== Network Packet Methods Demo ===\n");
    
    // Create base packet using constructor
    base_pkt = new(32'hC0A80001, 32'hC0A80002);  // 192.168.0.1 -> 192.168.0.2
    base_pkt.randomize();
    
    // Create TCP packet using constructor
    tcp_pkt = new(32'h08080808, 32'h08080404);   // 8.8.8.8 -> 8.8.4.4
    tcp_pkt.randomize();
    
    $display("\n--- Function Method Demo ---");
    $display("Base packet checksum: 0x%04h", base_pkt.calculate_checksum());
    $display("TCP packet checksum:  0x%04h", tcp_pkt.calculate_checksum());
    
    $display("\n--- Virtual Function Demo ---");
    $display("Base packet type: %s", base_pkt.get_packet_type());
    $display("TCP packet type:  %s", tcp_pkt.get_packet_type());
    
    $display("\n--- Static Method Demo ---");
    $display("Total packets created: %0d", 
             network_packet_base::get_total_packet_count());
    
    $display("\n--- Task Method Demo ---");
    base_pkt.display_packet_info();
    $display("");
    tcp_pkt.display_packet_info();
    
    $display("\n=== Demo Complete ===");
  end
  
endmodule
```

```systemverilog
// network_packet_methods_testbench.sv
// Chapter 9 Example 5: Network Packet Methods Testbench
// Tests different method types in network packet classes

module network_packet_methods_testbench;
  
  // Test handles
  network_packet_base packet_array[];
  tcp_packet         tcp_test_pkt;
  int                test_count = 0;
  
  // Test task for constructor method
  task test_constructor_method();
    network_packet_base pkt1, pkt2, pkt3;
    
    $display("\n=== Testing Constructor Method ===");
    
    // Test default constructor
    pkt1 = new();
    test_count++;
    
    // Test parameterized constructor
    pkt2 = new(32'hDEADBEEF, 32'hCAFEBABE);
    test_count++;
    
    // Test another constructor call
    pkt3 = new(32'h12345678, 32'h87654321);
    test_count++;
    
    $display("Constructor test completed. Packets created: %0d", test_count);
  endtask
  
  // Test function methods (pure functions, no time delay)
  task test_function_methods();
    network_packet_base test_pkt;
    bit [15:0] checksum1, checksum2;
    
    $display("\n=== Testing Function Methods ===");
    
    test_pkt = new(32'h11111111, 32'h22222222);
    test_pkt.packet_id = 8'hAB;
    test_pkt.packet_size = 16'h1234;
    
    // Call function method multiple times
    checksum1 = test_pkt.calculate_checksum();
    checksum2 = test_pkt.calculate_checksum();
    
    $display("Function method results:");
    $display("  Checksum 1: 0x%04h", checksum1);
    $display("  Checksum 2: 0x%04h", checksum2);
    $display("  Results match: %s", (checksum1 == checksum2) ? "PASS" : "FAIL");
  endtask
  
  // Test virtual methods (can be overridden)
  task test_virtual_methods();
    network_packet_base base_handle;
    tcp_packet         tcp_handle;
    
    $display("\n=== Testing Virtual Methods ===");
    
    // Create base packet
    base_handle = new();
    $display("Base packet type: %s", base_handle.get_packet_type());
    
    // Create TCP packet  
    tcp_handle = new();
    $display("TCP packet type:  %s", tcp_handle.get_packet_type());
    
    // Polymorphism test - base handle pointing to TCP object
    base_handle = tcp_handle;
    $display("Polymorphic call:  %s", base_handle.get_packet_type());
  endtask
  
  // Test static methods (class-level methods)
  task test_static_methods();
    int initial_count, final_count;
    network_packet_base temp_pkt1, temp_pkt2;
    
    $display("\n=== Testing Static Methods ===");
    
    initial_count = network_packet_base::get_total_packet_count();
    $display("Initial packet count: %0d", initial_count);
    
    // Create more packets
    temp_pkt1 = new();
    temp_pkt2 = new();
    
    final_count = network_packet_base::get_total_packet_count();
    $display("Final packet count: %0d", final_count);
    $display("Packets created in test: %0d", final_count - initial_count);
  endtask
  
  // Test task methods (can consume simulation time)
  task test_task_methods();
    tcp_packet test_tcp;
    realtime start_time, end_time;
    
    $display("\n=== Testing Task Methods ===");
    
    test_tcp = new(32'hAABBCCDD, 32'hEEFF1122);
    if (test_tcp.randomize() == 0) $display("Warning: TCP test packet randomization failed");
    
    start_time = $realtime;
    test_tcp.display_packet_info();  // This task has #1ns delay
    end_time = $realtime;
    
    $display("Task execution time: %0t", end_time - start_time);
  endtask
  
  // Main test sequence
  initial begin
    // Dump waves for debugging
    $dumpfile("network_packet_methods_testbench.vcd");
    $dumpvars(0, network_packet_methods_testbench);
    
    $display("=== Network Packet Methods Testbench ===");
    $display("Testing various method types in SystemVerilog classes\n");
    
    // Run individual tests
    test_constructor_method();
    test_function_methods();
    test_virtual_methods();
    test_static_methods();
    test_task_methods();
    
    $display("\n=== All Tests Completed ===");
    $display("Final packet count: %0d", 
             network_packet_base::get_total_packet_count());
    
    #10ns;  // Allow time for any pending operations
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
- Verilator: Walltime 36.835 s (elab=0.001, cvt=0.108, bld=36.449); cpu 0.047 s
on 1 threads; alloced 20.176 MB
=== Network Packet Methods Testbench ===
Testing various method types in SystemVerilog classes


=== Testing Constructor Method ===
Packet created: ID=0, Count=1
Packet created: ID=0, Count=2
Packet created: ID=0, Count=3
Constructor test completed. Packets created: 3

=== Testing Function Methods ===
Packet created: ID=0, Count=4
Function method results:
  Checksum 1: 0x00f1
  Checksum 2: 0x00f1
  Results match: PASS

=== Testing Virtual Methods ===
Packet created: ID=0, Count=5
Base packet type: BASE
Packet created: ID=0, Count=6
TCP packet type:  TCP
Polymorphic call:  TCP

=== Testing Static Methods ===
Initial packet count: 6
Packet created: ID=0, Count=7
Packet created: ID=0, Count=8
Final packet count: 8
Packets created in test: 2

=== Testing Task Methods ===
Packet created: ID=0, Count=9
=== Packet Info ===
ID: 0xb7, Size: 29011 bytes
Fro

0

### **Example 6: Method Return Types**
Examples of methods that return values vs. methods that perform actions without returning data.

In [7]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_6__method_return_types/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// method_return_demo.sv
// Demonstrates methods with return types vs. void methods

class calculator_class;
  int accumulator;
  
  // Constructor
  function new();
    accumulator = 0;
  endfunction
  
  // Method that returns a value (int return type)
  function int add_numbers(int a, int b);
    return a + b;
  endfunction
  
  // Method that returns a value and uses internal state
  function int get_accumulator();
    return accumulator;
  endfunction
  
  // Method that performs action without returning data (void)
  function void set_accumulator(int value);
    accumulator = value;
  endfunction
  
  // Method that performs action and updates state (void)
  function void add_to_accumulator(int value);
    accumulator += value;
  endfunction
  
  // Method that performs action for display (void)
  function void display_status();
    $display("Current accumulator value: %0d", accumulator);
  endfunction
  
  // Method that returns boolean result
  function bit is_positive();
    return (accumulator > 0);
  endfunction
  
  // Method that returns string for status
  function string get_status_string();
    if (accumulator > 0)
      return "POSITIVE";
    else if (accumulator < 0)
      return "NEGATIVE";
    else
      return "ZERO";
  endfunction
  
endclass

module method_return_demo;
  
  calculator_class calc;
  int result;
  bit status;
  string status_str;
  
  initial begin
    $display("=== Method Return Types Demo ===");
    $display();
    
    // Create calculator instance
    calc = new();
    
    // Using methods that return values
    result = calc.add_numbers(15, 25);
    $display("add_numbers(15, 25) returned: %0d", result);
    
    result = calc.get_accumulator();
    $display("get_accumulator() returned: %0d", result);
    
    // Using void methods (no return value)
    calc.set_accumulator(42);
    $display("Called set_accumulator(42) - no return value");
    
    calc.display_status();  // This method displays internally
    
    calc.add_to_accumulator(8);
    $display("Called add_to_accumulator(8) - no return value");
    
    // Using methods that return values again
    result = calc.get_accumulator();
    $display("get_accumulator() now returns: %0d", result);
    
    status = calc.is_positive();
    $display("is_positive() returned: %0b", status);
    
    status_str = calc.get_status_string();
    $display("get_status_string() returned: %s", status_str);
    
    $display();
    $display("=== Testing with negative value ===");
    calc.set_accumulator(-10);
    calc.display_status();
    
    status = calc.is_positive();
    $display("is_positive() returned: %0b", status);
    
    status_str = calc.get_status_string();
    $display("get_status_string() returned: %s", status_str);
    
  end
  
endmodule
```

```systemverilog
// method_return_demo_testbench.sv
// Testbench for method return types demonstration

module method_return_testbench;
  
  // Instantiate design under test
  method_return_demo METHOD_RETURN_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("method_return_testbench.vcd");
    $dumpvars(0, method_return_testbench);
    
    // Wait for design to complete
    #1;
    
    $display("Hello from method return types testbench!");
    $display("Design execution completed successfully.");
    $display();
    
    // Additional test scenarios in testbench
    $display("=== Additional Testbench Verification ===");
    test_return_value_methods();
    test_void_methods();
    
    $display("=== Testbench completed ===");
    $finish;
  end
  
  // Task to test methods that return values
  task test_return_value_methods();
    calculator_class test_calc;
    int math_result;
    bit bool_result;
    string str_result;
    
    $display("Testing methods with return values:");
    
    test_calc = new();
    
    // Test mathematical operation
    math_result = test_calc.add_numbers(100, 200);
    $display("  add_numbers(100, 200) = %0d", math_result);
    
    // Test getter method
    test_calc.set_accumulator(75);
    math_result = test_calc.get_accumulator();
    $display("  get_accumulator() = %0d", math_result);
    
    // Test boolean return
    bool_result = test_calc.is_positive();
    $display("  is_positive() = %0b", bool_result);
    
    // Test string return
    str_result = test_calc.get_status_string();
    $display("  get_status_string() = %s", str_result);
    
    $display();
  endtask
  
  // Task to test void methods
  task test_void_methods();
    calculator_class test_calc;
    
    $display("Testing void methods (no return value):");
    
    test_calc = new();
    
    // These methods don't return values, just perform actions
    $display("  Calling set_accumulator(99)...");
    test_calc.set_accumulator(99);
    
    $display("  Calling add_to_accumulator(-50)...");
    test_calc.add_to_accumulator(-50);
    
    $display("  Calling display_status()...");
    test_calc.display_status();
    
    $display("  All void methods executed successfully.");
    $display();
  endtask
  
endmodule
```

Verilator Simulation Output:
=== Method Return Types Demo ===

add_numbers(15, 25) returned: 40
get_accumulator() returned: 0
Called set_accumulator(42) - no return value
Current accumulator value: 42
Called add_to_accumulator(8) - no return value
get_accumulator() now returns: 50
is_positive() returned: 1
get_status_string() returned: POSITIVE

=== Testing with negative value ===
Current accumulator value: -10
is_positive() returned: 0
get_status_string() returned: NEGATIVE
Hello from method return types testbench!
Design execution completed successfully.

=== Additional Testbench Verification ===
Testing methods with return values:
  add_numbers(100, 200) = 300
  get_accumulator() = 75
  is_positive() = 1
  get_status_string() = POSITIVE

Testing void methods (no return value):
  Calling set_accumulator(99)...
  Calling add_to_accumulator(-50)...
  Calling display_status()...
Current accumulator value: 49
  All void methods executed successfully.

=== Testbench completed ===
Process 

0

## Object Creation and Destruction

Objects are instances of classes created using the `new()` constructor. SystemVerilog handles memory management automatically.

### Object Creation

```systemverilog
class ConfigBlock;
    bit [31:0] base_addr;
    bit [7:0] version;
    bit enable;
    
    function new(bit [31:0] addr = 32'h1000);
        base_addr = addr;
        version = 8'h01;
        enable = 1'b1;
    endfunction
    
    function void configure(bit [31:0] addr, bit en);
        base_addr = addr;
        enable = en;
    endfunction
endclass

// Usage example
module test_objects;
    ConfigBlock cfg1, cfg2, cfg3;
    
    initial begin
        // Create objects
        cfg1 = new();                    // Use default constructor
        cfg2 = new(32'h2000);           // Pass parameter to constructor
        cfg3 = new(32'h3000);
        
        // Use objects
        cfg1.configure(32'h1500, 1'b0);
        cfg2.version = 8'h02;
        
        // Display object contents
        $display("Config 1: Addr=%h, Ver=%h, En=%b", 
                 cfg1.base_addr, cfg1.version, cfg1.enable);
        $display("Config 2: Addr=%h, Ver=%h, En=%b", 
                 cfg2.base_addr, cfg2.version, cfg2.enable);
    end
endmodule
```

### Object Assignment and Copying

```systemverilog
class DataBuffer;
    bit [7:0] buffer[];
    int size;
    
    function new(int sz = 16);
        size = sz;
        buffer = new[sz];
    endfunction
    
    // Deep copy method
    function DataBuffer copy();
        DataBuffer new_buf = new(size);
        new_buf.buffer = new[size];
        foreach(buffer[i]) new_buf.buffer[i] = buffer[i];
        return new_buf;
    endfunction
    
    function void fill_random();
        foreach(buffer[i]) buffer[i] = $random;
    endfunction
endclass

// Usage
module test_copy;
    DataBuffer buf1, buf2, buf3;
    
    initial begin
        buf1 = new(32);
        buf1.fill_random();
        
        buf2 = buf1;        // Shallow copy (both handles point to same object)
        buf3 = buf1.copy(); // Deep copy (creates new object)
        
        // Modify original
        buf1.buffer[0] = 8'hFF;
        
        // buf2 sees the change, buf3 doesn't
        $display("buf1[0] = %h", buf1.buffer[0]); // FF
        $display("buf2[0] = %h", buf2.buffer[0]); // FF (same object)
        $display("buf3[0] = %h", buf3.buffer[0]); // original value
    end
endmodule
```

### **Example 7: Configuration Block Objects**
Simple example of creating multiple objects with different constructor parameters.

In [10]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_7__configuration_block_objects/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// packet_config_design.sv
class packet_config;
  // Configuration parameters
  int packet_size;
  int num_packets;
  string packet_type;
  bit enable_crc;
  
  // Constructor with default parameters
  function new(int size = 64, 
               int count = 10, 
               string p_type = "DATA", 
               bit crc = 1'b1);
    packet_size = size;
    num_packets = count;
    packet_type = p_type;
    enable_crc = crc;
    $display("Config created: Size=%0d, Count=%0d, Type=%s, CRC=%b",
             packet_size, num_packets, packet_type, enable_crc);
  endfunction
  
  // Display configuration
  function void display_config();
    $display("--- Packet Configuration ---");
    $display("Packet Size: %0d bytes", packet_size);
    $display("Number of Packets: %0d", num_packets);
    $display("Packet Type: %s", packet_type);
    $display("CRC Enabled: %s", enable_crc ? "YES" : "NO");
    $display("---------------------------");
  endfunction
endclass

module packet_config_module;
  packet_config cfg1, cfg2, cfg3, cfg4;
  
  initial begin
    $display("=== Configuration Block Objects Example ===");
    $display();
    
    // Create different configuration objects
    $display("Creating default configuration...");
    cfg1 = new();  // Use all defaults
    cfg1.display_config();
    $display();
    
    $display("Creating small packet configuration...");
    cfg2 = new(.size(32), .count(5));  // Small packets, few count
    cfg2.display_config();
    $display();
    
    $display("Creating control packet configuration...");
    cfg3 = new(.size(16), .count(3), .p_type("CTRL"), .crc(1'b0));
    cfg3.display_config();
    $display();
    
    $display("Creating large data configuration...");
    cfg4 = new(.size(1024), .count(100), .p_type("BULK_DATA"));
    cfg4.display_config();
    $display();
    
    $display("=== All configurations created successfully ===");
  end
endmodule
```

```systemverilog
// packet_config_design_testbench.sv
module packet_config_testbench;
  // Instantiate design under test
  packet_config_module PACKET_CONFIG_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("packet_config_testbench.vcd");
    $dumpvars(0, packet_config_testbench);
    
    // Wait for design to complete
    #10;
    
    $display();
    $display("=== Testbench: Configuration Objects Test Complete ===");
    $display("Multiple packet configurations created with different");
    $display("constructor parameters demonstrating flexible object");
    $display("initialization patterns.");
    $display();
    
    // Finish simulation
    $finish;
  end
endmodule
```

Verilator Simulation Output:
=== Configuration Block Objects Example ===

Creating default configuration...
Config created: Size=64, Count=10, Type=DATA, CRC=1
--- Packet Configuration ---
Packet Size: 64 bytes
Number of Packets: 10
Packet Type: DATA
CRC Enabled: YES
---------------------------

Creating small packet configuration...
Config created: Size=32, Count=5, Type=DATA, CRC=1
--- Packet Configuration ---
Packet Size: 32 bytes
Number of Packets: 5
Packet Type: DATA
CRC Enabled: YES
---------------------------

Creating control packet configuration...
Config created: Size=16, Count=3, Type=CTRL, CRC=0
--- Packet Configuration ---
Packet Size: 16 bytes
Number of Packets: 3
Packet Type: CTRL
CRC Enabled:  NO
---------------------------

Creating large data configuration...
Config created: Size=1024, Count=100, Type=BULK_DATA, CRC=1
--- Packet Configuration ---
Packet Size: 1024 bytes
Number of Packets: 100
Packet Type: BULK_DATA
CRC Enabled: YES
---------------------------

=== All

0

### **Example 8: Object Assignment Demo**
Shows the difference between shallow copy (handle assignment) and deep copy methods.

In [12]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_8__object_assignment_demo/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// object_assignment_demo.sv
class data_packet;
  int packet_id;
  string payload;
  int priority_level;
  
  function new(int id = 0, string data = "default", int prio = 1);
    packet_id = id;
    payload = data;
    priority_level = prio;
  endfunction
  
  // Deep copy method
  function data_packet deep_copy();
    data_packet copied_packet = new();
    copied_packet.packet_id = this.packet_id;
    copied_packet.payload = this.payload;
    copied_packet.priority_level = this.priority_level;
    return copied_packet;
  endfunction
  
  // Display packet contents
  function void display_packet(string prefix = "");
    $display("%sPacket ID: %0d, Payload: %s, Priority: %0d", 
             prefix, packet_id, payload, priority_level);
  endfunction
endclass

module object_assignment_demo();
  initial begin
    $display("=== Object Assignment Demo ===");
    $display();
  end
endmodule
```

```systemverilog
// object_assignment_demo_testbench.sv
module object_assignment_testbench;
  object_assignment_demo DESIGN_INSTANCE();
  
  initial begin
    data_packet original_packet;
    data_packet shallow_copy_packet;
    data_packet deep_copy_packet;
    
    // Dump waves
    $dumpfile("object_assignment_testbench.vcd");
    $dumpvars(0, object_assignment_testbench);
    
    $display("=== Object Assignment Demo ===");
    $display();
    
    // Create original packet
    original_packet = new(101, "Important Data", 5);
    $display("1. Original packet created:");
    original_packet.display_packet("   ");
    $display();
    
    // Shallow copy (handle assignment)
    $display("2. Performing SHALLOW COPY (handle assignment):");
    shallow_copy_packet = original_packet;  // Same object, different handle
    $display("   shallow_copy_packet = original_packet;");
    shallow_copy_packet.display_packet("   Shallow copy: ");
    $display();
    
    // Modify original packet
    $display("3. Modifying original packet payload:");
    original_packet.payload = "Modified Data";
    $display("   original_packet.payload = \"Modified Data\";");
    $display("   Original packet after modification:");
    original_packet.display_packet("   ");
    $display("   Shallow copy packet (same object!):");
    shallow_copy_packet.display_packet("   ");
    $display("   --> Both changed because they point to same object!");
    $display();
    
    // Deep copy demonstration
    $display("4. Performing DEEP COPY (creating new object):");
    original_packet.payload = "Original Data";  // Reset for demo
    deep_copy_packet = original_packet.deep_copy();
    $display("   deep_copy_packet = original_packet.deep_copy();");
    deep_copy_packet.display_packet("   Deep copy: ");
    $display();
    
    // Modify original packet again
    $display("5. Modifying original packet priority:");
    original_packet.priority_level = 10;
    $display("   original_packet.priority_level = 10;");
    $display("   Original packet after modification:");
    original_packet.display_packet("   ");
    $display("   Deep copy packet (independent object!):");
    deep_copy_packet.display_packet("   ");
    $display("   --> Deep copy unchanged - it's a separate object!");
    $display();
    
    $display("=== Summary ===");
    $display("Shallow copy: Same object, different handle names");
    $display("Deep copy:    New object with copied values");
    $display();
    
    #1;
  end
endmodule
```

Verilator Simulation Output:
=== Object Assignment Demo ===

=== Object Assignment Demo ===

1. Original packet created:
   Packet ID: 101, Payload: Important Data, Priority: 5

2. Performing SHALLOW COPY (handle assignment):
   shallow_copy_packet = original_packet;
   Shallow copy: Packet ID: 101, Payload: Important Data, Priority: 5

3. Modifying original packet payload:
   original_packet.payload = "Modified Data";
   Original packet after modification:
   Packet ID: 101, Payload: Modified Data, Priority: 5
   Shallow copy packet (same object!):
   Packet ID: 101, Payload: Modified Data, Priority: 5
   --> Both changed because they point to same object!

4. Performing DEEP COPY (creating new object):
   deep_copy_packet = original_packet.deep_copy();
   Deep copy: Packet ID: 101, Payload: Original Data, Priority: 5

5. Modifying original packet priority:
   original_packet.priority_level = 10;
   Original packet after modification:
   Packet ID: 101, Payload: Original Data, Priorit

0

### **Example 9: Buffer Management**
Demonstrates object creation, modification, and memory management concepts.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_9__buffer_management/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// circular_buffer_manager.sv
module circular_buffer_manager #(
  parameter int BUFFER_SIZE = 8,
  parameter int DATA_WIDTH = 8
)(
  input  logic                    clk,
  input  logic                    rst_n,
  input  logic                    write_enable,
  input  logic                    read_enable,
  input  logic [DATA_WIDTH-1:0]   write_data,
  output logic [DATA_WIDTH-1:0]   read_data,
  output logic                    buffer_full,
  output logic                    buffer_empty,
  output logic [$clog2(BUFFER_SIZE):0] buffer_count
);

  // Internal buffer memory
  logic [DATA_WIDTH-1:0] buffer_memory [BUFFER_SIZE-1:0];
  logic [$clog2(BUFFER_SIZE)-1:0] write_pointer;
  logic [$clog2(BUFFER_SIZE)-1:0] read_pointer;
  logic [$clog2(BUFFER_SIZE):0] item_count;

  // Buffer status flags
  assign buffer_full = (item_count == ($clog2(BUFFER_SIZE)+1)'(BUFFER_SIZE));
  assign buffer_empty = (item_count == 0);
  assign buffer_count = item_count;

  // Write operation
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      write_pointer <= 0;
      item_count <= 0;
      // Initialize buffer memory
      for (int i = 0; i < BUFFER_SIZE; i++) begin
        buffer_memory[i] <= 0;
      end
    end else begin
      if (write_enable && !buffer_full) begin
        buffer_memory[write_pointer] <= write_data;
        if (write_pointer == ($clog2(BUFFER_SIZE))'(BUFFER_SIZE - 1)) begin
          write_pointer <= 0;
        end else begin
          write_pointer <= write_pointer + 1'b1;
        end
        if (!read_enable || buffer_empty) begin
          item_count <= item_count + 1'b1;
        end
      end else if (read_enable && !buffer_empty && !write_enable) begin
        item_count <= item_count - 1'b1;
      end
    end
  end

  // Read operation
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      read_pointer <= 0;
      read_data <= 0;
    end else begin
      if (read_enable && !buffer_empty) begin
        read_data <= buffer_memory[read_pointer];
        if (read_pointer == ($clog2(BUFFER_SIZE))'(BUFFER_SIZE - 1)) begin
          read_pointer <= 0;
        end else begin
          read_pointer <= read_pointer + 1'b1;
        end
      end
    end
  end

  // Display buffer contents for debugging
  initial begin
    $display("Buffer Manager initialized with size: %0d", BUFFER_SIZE);
    $display("Data width: %0d bits", DATA_WIDTH);
  end

endmodule
```

```systemverilog
// circular_buffer_manager_testbench.sv
module buffer_manager_testbench;

  // Testbench parameters
  parameter int BUFFER_SIZE = 4;
  parameter int DATA_WIDTH = 8;
  parameter int CLK_PERIOD = 10;

  // Testbench signals
  logic                    clk;
  logic                    rst_n;
  logic                    write_enable;
  logic                    read_enable;
  logic [DATA_WIDTH-1:0]   write_data;
  logic [DATA_WIDTH-1:0]   read_data;
  logic                    buffer_full;
  logic                    buffer_empty;
  logic [$clog2(BUFFER_SIZE):0] buffer_count;

  // Clock generation
  always #(CLK_PERIOD/2) clk = ~clk;

  // Instantiate design under test
  circular_buffer_manager #(
    .BUFFER_SIZE(BUFFER_SIZE),
    .DATA_WIDTH(DATA_WIDTH)
  ) BUFFER_MANAGER_INSTANCE (
    .clk(clk),
    .rst_n(rst_n),
    .write_enable(write_enable),
    .read_enable(read_enable),
    .write_data(write_data),
    .read_data(read_data),
    .buffer_full(buffer_full),
    .buffer_empty(buffer_empty),
    .buffer_count(buffer_count)
  );

  // Test sequence
  initial begin
    // Initialize VCD dump
    $dumpfile("buffer_manager_testbench.vcd");
    $dumpvars(0, buffer_manager_testbench);

    // Initialize signals
    clk = 0;
    rst_n = 0;
    write_enable = 0;
    read_enable = 0;
    write_data = 0;

    $display("=== Buffer Management Test Started ===");
    $display();

    // Reset sequence
    #(CLK_PERIOD * 2);
    rst_n = 1;
    #(CLK_PERIOD);

    // Test 1: Fill buffer with data
    $display("Test 1: Writing data to buffer");
    for (int i = 0; i < BUFFER_SIZE; i++) begin
      write_data = 8'hA0 + 8'(i);
      write_enable = 1;
      #(CLK_PERIOD);
      write_enable = 0;
      $display("  Written: 0x%02X, Count: %0d, Full: %b", 
               write_data, buffer_count, buffer_full);
      #(CLK_PERIOD);
    end

    // Test 2: Try to write when buffer is full
    $display("\nTest 2: Attempting to write when buffer is full");
    write_data = 8'hFF;
    write_enable = 1;
    #(CLK_PERIOD);
    write_enable = 0;
    $display("  Write attempt: 0x%02X, Count: %0d, Full: %b", 
             write_data, buffer_count, buffer_full);
    #(CLK_PERIOD);

    // Test 3: Read data from buffer
    $display("\nTest 3: Reading data from buffer");
    for (int i = 0; i < BUFFER_SIZE; i++) begin
      read_enable = 1;
      #(CLK_PERIOD);
      read_enable = 0;
      $display("  Read: 0x%02X, Count: %0d, Empty: %b", 
               read_data, buffer_count, buffer_empty);
      #(CLK_PERIOD);
    end

    // Test 4: Try to read when buffer is empty
    $display("\nTest 4: Attempting to read when buffer is empty");
    read_enable = 1;
    #(CLK_PERIOD);
    read_enable = 0;
    $display("  Read attempt: 0x%02X, Count: %0d, Empty: %b", 
             read_data, buffer_count, buffer_empty);
    #(CLK_PERIOD);

    // Test 5: Simultaneous read/write operations
    $display("\nTest 5: Simultaneous read/write operations");
    write_data = 8'hBB;
    write_enable = 1;
    #(CLK_PERIOD);
    write_enable = 0;
    $display("  Written: 0x%02X, Count: %0d", write_data, buffer_count);
    
    read_enable = 1;
    write_enable = 1;
    write_data = 8'hCC;
    #(CLK_PERIOD);
    write_enable = 0;
    read_enable = 0;
    $display("  Simultaneous R/W - Read: 0x%02X, Written: 0x%02X, Count: %0d", 
             read_data, write_data, buffer_count);

    #(CLK_PERIOD * 2);
    $display();
    $display("=== Buffer Management Test Completed ===");
    $display("Final buffer state - Count: %0d, Full: %b, Empty: %b", 
             buffer_count, buffer_full, buffer_empty);

    $finish;
  end

endmodule
```

Verilator Simulation Output:
Buffer Manager initialized with size: 4
Data width: 8 bits
=== Buffer Management Test Started ===

Test 1: Writing data to buffer
  Written: 0xa0, Count: 1, Full: 0
  Written: 0xa1, Count: 2, Full: 0
  Written: 0xa2, Count: 3, Full: 0
  Written: 0xa3, Count: 4, Full: 1

Test 2: Attempting to write when buffer is full
  Write attempt: 0xff, Count: 4, Full: 1

Test 3: Reading data from buffer
  Read: 0xa0, Count: 3, Empty: 0
  Read: 0xa1, Count: 2, Empty: 0
  Read: 0xa2, Count: 1, Empty: 0
  Read: 0xa3, Count: 0, Empty: 1

Test 4: Attempting to read when buffer is empty
  Read attempt: 0xa3, Count: 0, Empty: 1

Test 5: Simultaneous read/write operations
  Written: 0xbb, Count: 1
  Simultaneous R/W - Read: 0xbb, Written: 0xcc, Count: 1

=== Buffer Management Test Completed ===
Final buffer state - Count: 1, Full: 0, Empty: 0
Process finished with return code: 0
Removing Chapter_9_examples/example_9__buffer_management/obj_dir directory...
Chapter_9_examples/exa

0

## The `this` Keyword

The `this` keyword refers to the current object instance and is used to resolve naming conflicts or for explicit reference.

```systemverilog
class Counter;
    int count;
    string name;
    
    function new(string name, int count = 0);
        this.name = name;    // Distinguish parameter from property
        this.count = count;  // Explicit reference to object property
    endfunction
    
    function void increment(int count = 1);
        this.count += count; // Use this to access object property
    endfunction
    
    function Counter get_copy();
        Counter copy = new(this.name, this.count);
        return copy;
    endfunction
    
    function void compare_with(Counter other);
        if (this.count > other.count)
            $display("%s (%0d) > %s (%0d)", 
                     this.name, this.count, other.name, other.count);
        else if (this.count < other.count)
            $display("%s (%0d) < %s (%0d)", 
                     this.name, this.count, other.name, other.count);
        else
            $display("%s and %s have equal counts (%0d)", 
                     this.name, other.name, this.count);
    endfunction
endclass

// Usage
module test_this;
    Counter c1, c2;
    
    initial begin
        c1 = new("Counter1", 5);
        c2 = new("Counter2", 3);
        
        c1.increment(2);
        c1.compare_with(c2);
        
        c2 = c1.get_copy();
        c2.name = "Counter2_copy";
        c1.compare_with(c2);
    end
endmodule
```

### **Example 10: Counter Class with this**
Class using `this` keyword to resolve naming conflicts between parameters and properties.

In [9]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_10__counter_class_with_this/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// counter_class_with_this.sv

// Package containing the Counter class
package counter_pkg;

  // Counter class demonstrating 'this' keyword usage
  class Counter;
    int count;        // Class property
    int step;         // Class property

    // Constructor with parameter names matching property names
    function new(int count = 0, int step = 1);
      this.count = count;    // Use 'this' to distinguish property from parameter
      this.step = step;      // Use 'this' to distinguish property from parameter
    endfunction

    // Method to increment counter
    function void increment();
      this.count += this.step;
    endfunction

    // Method to decrement counter
    function void decrement();
      this.count -= this.step;
    endfunction

    // Method to set count value
    function void set_count(int count);
      this.count = count;    // Use 'this' to resolve naming conflict
    endfunction

    // Method to set step value
    function void set_step(int step);
      this.step = step;      // Use 'this' to resolve naming conflict
    endfunction

    // Method to get current count
    function int get_count();
      return this.count;
    endfunction

    // Method to display counter state
    function void display();
      $display("Counter: count=%0d, step=%0d", this.count, this.step);
    endfunction

  endclass

endpackage

// Module using the Counter class
module counter_class_module();

  // Import the Counter class from package
  import counter_pkg::*;

  // Test the Counter class
  initial begin
    Counter my_counter;
    
    $display("=== Counter Class with 'this' Keyword Example ===");
    $display();
    
    // Create counter instance
    my_counter = new(5, 2);
    $display("Initial counter:");
    my_counter.display();
    $display();
    
    // Test increment
    my_counter.increment();
    $display("After increment:");
    my_counter.display();
    $display();
    
    // Test decrement
    my_counter.decrement();
    $display("After decrement:");
    my_counter.display();
    $display();
    
    // Test set methods with naming conflicts
    my_counter.set_count(10);
    my_counter.set_step(3);
    $display("After setting count=10, step=3:");
    my_counter.display();
    $display();
    
    // Test multiple increments
    my_counter.increment();
    my_counter.increment();
    $display("After two increments:");
    my_counter.display();
    $display();
    
    $display("Final count value: %0d", my_counter.get_count());
  end

endmodule
```

```systemverilog
// counter_class_with_this_testbench.sv
module counter_class_testbench;

  // Import the Counter class from package
  import counter_pkg::*;

  // Instantiate the design under test
  counter_class_module DUT();

  initial begin
    // Dump waves for simulation
    $dumpfile("counter_class_testbench.vcd");
    $dumpvars(0, counter_class_testbench);
    
    $display("=== Testbench for Counter Class with 'this' ===");
    $display();
    
    // Wait for design to complete
    #100;
    
    $display();
    $display("=== Additional Testbench Verification ===");
    
    // Create additional counter instances to verify class functionality
    begin
      Counter test_counter1, test_counter2;
      
      // Test different initialization values
      test_counter1 = new(0, 5);
      test_counter2 = new(100, 10);
      
      $display("Test Counter 1 (start=0, step=5):");
      test_counter1.display();
      test_counter1.increment();
      test_counter1.increment();
      test_counter1.display();
      $display();
      
      $display("Test Counter 2 (start=100, step=10):");
      test_counter2.display();
      test_counter2.decrement();
      test_counter2.display();
      $display();
      
      // Verify get_count method
      $display("Counter 1 final value: %0d", test_counter1.get_count());
      $display("Counter 2 final value: %0d", test_counter2.get_count());
    end
    
    $display();
    $display("=== Testbench Complete ===");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Counter Class with 'this' Keyword Example ===

Initial counter:
Counter: count=5, step=2

After increment:
Counter: count=7, step=2

After decrement:
Counter: count=5, step=2

After setting count=10, step=3:
Counter: count=10, step=3

After two increments:
Counter: count=16, step=3

Final count value: 16
=== Testbench for Counter Class with 'this' ===


=== Additional Testbench Verification ===
Test Counter 1 (start=0, step=5):
Counter: count=0, step=5
Counter: count=10, step=5

Test Counter 2 (start=100, step=10):
Counter: count=100, step=10
Counter: count=90, step=10

Counter 1 final value: 10
Counter 2 final value: 90

=== Testbench Complete ===
Process finished with return code: 0
Removing Chapter_9_examples/example_10__counter_class_with_this/obj_dir directory...
Chapter_9_examples/example_10__counter_class_with_this/obj_dir removed successfully.


0

### **Example 11: Object Self-Reference**
Examples of methods that return references to the current object using `this`.

In [12]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_11__object_self_reference/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// object_self_reference.sv
// Example demonstrating object self-reference using 'this' keyword

class ConfigBuilder;
  string config_name;
  int timeout_value;
  bit debug_enable;
  
  // Constructor
  function new(string name = "default_config");
    this.config_name = name;
    this.timeout_value = 100;
    this.debug_enable = 0;
  endfunction
  
  // Method that sets timeout and returns self-reference for chaining
  function ConfigBuilder set_timeout(int timeout);
    this.timeout_value = timeout;
    return this;  // Return reference to current object
  endfunction
  
  // Method that sets debug mode and returns self-reference for chaining
  function ConfigBuilder set_debug(bit enable);
    this.debug_enable = enable;
    return this;  // Return reference to current object
  endfunction
  
  // Method that sets name and returns self-reference for chaining
  function ConfigBuilder set_name(string name);
    this.config_name = name;
    return this;  // Return reference to current object
  endfunction
  
  // Method to display current configuration
  function void display_config();
    $display("Configuration: %s", this.config_name);
    $display("  Timeout: %0d", this.timeout_value);
    $display("  Debug: %s", this.debug_enable ? "ON" : "OFF");
  endfunction
  
  // Method that returns a copy of this object
  function ConfigBuilder clone();
    ConfigBuilder copy = new(this.config_name);
    copy.timeout_value = this.timeout_value;
    copy.debug_enable = this.debug_enable;
    return copy;
  endfunction
  
endclass

module design_module_name;
  
  initial begin
    ConfigBuilder cfg1, cfg2, cfg3;
    
    $display("=== Object Self-Reference Example ===");
    $display();
    
    // Create initial configuration
    cfg1 = new("basic_config");
    $display("Initial configuration:");
    cfg1.display_config();
    $display();
    
    // Demonstrate method chaining using self-reference
    $display("Method chaining example:");
    cfg2 = cfg1.set_name("chained_config")
              .set_timeout(500)
              .set_debug(1);
    cfg2.display_config();
    $display();
    
    // Demonstrate cloning (another use of self-reference)
    $display("Cloning example:");
    cfg3 = cfg2.clone();
    void'(cfg3.set_name("cloned_config"));  // Explicit void cast
    cfg3.display_config();
    $display();
    
    // Verify original object is unchanged
    $display("Original object after cloning:");
    cfg2.display_config();
    $display();
    
  end
  
endmodule
```

```systemverilog
// object_self_reference_testbench.sv
// Testbench for object self-reference example

module test_bench_module;
  
  // Instantiate design under test
  design_module_name DESIGN_INSTANCE_NAME();
  
  initial begin
    // Dump waves
    $dumpfile("test_bench_module.vcd");
    $dumpvars(0, test_bench_module);
    
    #1;  // Wait for a time unit
    
    $display("=== Testbench Additional Tests ===");
    $display();
    
    // Additional test to verify self-reference behavior
    begin
      ConfigBuilder test_cfg;
      ConfigBuilder returned_ref;
      
      test_cfg = new("test_reference");
      $display("Testing self-reference return:");
      test_cfg.display_config();
      $display();
      
      // Call method and capture returned reference
      returned_ref = test_cfg.set_timeout(999);
      
      // Verify that returned reference points to same object
      $display("Original object after method call:");
      test_cfg.display_config();
      $display();
      
      $display("Returned reference object:");
      returned_ref.display_config();
      $display();
      
      // Modify through returned reference
      void'(returned_ref.set_debug(1));  // Explicit void cast
      $display("Original object after modifying through returned reference:");
      test_cfg.display_config();
      $display();
      
    end
    
    $display("Hello from testbench!");
    $display();
    
  end
  
endmodule
```

Verilator Simulation Output:
=== Object Self-Reference Example ===

Initial configuration:
Configuration: basic_config
  Timeout: 100
  Debug: OFF

Method chaining example:
Configuration: chained_config
  Timeout: 500
  Debug:  ON

Cloning example:
Configuration: cloned_config
  Timeout: 500
  Debug:  ON

Original object after cloning:
Configuration: chained_config
  Timeout: 500
  Debug:  ON

=== Testbench Additional Tests ===

Testing self-reference return:
Configuration: test_reference
  Timeout: 100
  Debug: OFF

Original object after method call:
Configuration: test_reference
  Timeout: 999
  Debug: OFF

Returned reference object:
Configuration: test_reference
  Timeout: 999
  Debug: OFF

Original object after modifying through returned reference:
Configuration: test_reference
  Timeout: 999
  Debug:  ON

Hello from testbench!
Process finished with return code: 0
Removing Chapter_9_examples/example_11__object_self_reference/obj_dir directory...
Chapter_9_examples/example_11__objec

0

### **Example 12: Object Comparison**
Method that compares the current object with another object using `this`.

In [14]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_12__object_comparison/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// object_comparison_design.sv
class student_record;
  string name;
  int age;
  int grade;
  
  function new(string n = "Unknown", int a = 0, int g = 0);
    this.name = n;
    this.age = a;
    this.grade = g;
  endfunction
  
  // Method to compare current object with another using 'this'
  function bit is_equal(student_record other);
    if (other == null) return 0;
    
    return (this.name == other.name && 
            this.age == other.age && 
            this.grade == other.grade);
  endfunction
  
  // Method to check if current student is older than another
  function bit is_older_than(student_record other);
    if (other == null) return 0;
    
    return (this.age > other.age);
  endfunction
  
  // Method to display student info
  function void display();
    $display("Student: %s, Age: %0d, Grade: %0d", 
             this.name, this.age, this.grade);
  endfunction
  
endclass

module object_comparison_module();
  student_record student1, student2, student3;
  
  initial begin
    $display("=== Object Comparison Example ===");
    $display();
    
    // Create student objects
    student1 = new("Alice", 20, 85);
    student2 = new("Bob", 22, 90);
    student3 = new("Alice", 20, 85);  // Same as student1
    
    // Display students
    $display("Created students:");
    student1.display();
    student2.display();
    student3.display();
    $display();
    
    // Compare students using 'this'
    $display("Comparison results:");
    $display("Student1 == Student2: %s", 
             student1.is_equal(student2) ? "TRUE" : "FALSE");
    $display("Student1 == Student3: %s", 
             student1.is_equal(student3) ? "TRUE" : "FALSE");
    $display();
    
    // Age comparison using 'this'
    $display("Age comparison results:");
    $display("Student1 older than Student2: %s", 
             student1.is_older_than(student2) ? "TRUE" : "FALSE");
    $display("Student2 older than Student1: %s", 
             student2.is_older_than(student1) ? "TRUE" : "FALSE");
    $display();
  end
  
endmodule
```

```systemverilog
// object_comparison_design_testbench.sv
module object_comparison_testbench;
  
  // Instantiate design under test
  object_comparison_module COMPARISON_INSTANCE();
  
  initial begin
    // Dump waves
    $dumpfile("object_comparison_testbench.vcd");
    $dumpvars(0, object_comparison_testbench);
    
    #1;  // Wait for design to complete
    
    $display("=== Testbench Additional Tests ===");
    $display();
    
    // Additional test with null comparison
    begin
      student_record test_student, null_student;
      
      test_student = new("Charlie", 19, 78);
      null_student = null;
      
      $display("Testing null comparison:");
      test_student.display();
      $display("Test student == null: %s", 
               test_student.is_equal(null_student) ? "TRUE" : "FALSE");
      $display("Test student older than null: %s", 
               test_student.is_older_than(null_student) ? "TRUE" : "FALSE");
      $display();
    end
    
    $display("Object comparison testbench completed!");
    $display();
    
    #10;  // Additional wait time
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Object Comparison Example ===

Created students:
Student: Alice, Age: 20, Grade: 85
Student: Bob, Age: 22, Grade: 90
Student: Alice, Age: 20, Grade: 85

Comparison results:
Student1 == Student2: FALSE
Student1 == Student3:  TRUE

Age comparison results:
Student1 older than Student2: FALSE
Student2 older than Student1:  TRUE

=== Testbench Additional Tests ===

Testing null comparison:
Student: Charlie, Age: 19, Grade: 78
Test student == null: FALSE
Test student older than null: FALSE

Object comparison testbench completed!

Process finished with return code: 0
Removing Chapter_9_examples/example_12__object_comparison/obj_dir directory...
Chapter_9_examples/example_12__object_comparison/obj_dir removed successfully.


0

## Class Scope and Lifetime

Class scope defines the visibility of class members, while lifetime determines when objects are created and destroyed.

### Access Control

```systemverilog
class SecureData;
    // Public members (default)
    string public_info;
    
    // Protected members (accessible in derived classes)
    protected bit [31:0] protected_key;
    
    // Local members (private to this class)
    local bit [127:0] private_data;
    local bit [7:0] secret_code;
    
    function new(string info = "default");
        public_info = info;
        protected_key = $random;
        private_data = {$random, $random, $random, $random};
        secret_code = 8'hA5;
    endfunction
    
    // Public method to access private data
    function bit [31:0] get_hash();
        return private_data[31:0] ^ protected_key ^ {24'b0, secret_code};
    endfunction
    
    // Protected method for derived classes
    protected function bit [31:0] get_protected_key();
        return protected_key;
    endfunction
    
    // Local method (private)
    local function bit verify_secret(bit [7:0] code);
        return (code == secret_code);
    endfunction
endclass

// Extended class demonstrating scope
class ExtendedSecureData extends SecureData;
    function new(string info = "extended");
        super.new(info);
    endfunction
    
    function void show_protected();
        // Can access protected members
        $display("Protected key: %h", protected_key);
        $display("Using protected method: %h", get_protected_key());
        
        // Cannot access local/private members
        // $display("Secret: %h", secret_code); // Error!
    endfunction
endclass
```

### Object Lifetime

```systemverilog
class Resource;
    static int instance_count = 0;
    int id;
    string name;
    
    function new(string name);
        instance_count++;
        id = instance_count;
        this.name = name;
        $display("Resource %0d (%s) created", id, name);
    endfunction
    
    // Destructor-like method (called explicitly)
    function void cleanup();
        $display("Resource %0d (%s) cleaned up", id, name);
        // Custom cleanup code here
    endfunction
    
    static function int get_instance_count();
        return instance_count;
    endfunction
endclass

module test_lifetime;
    Resource res1, res2;
    
    initial begin
        $display("Initial count: %0d", Resource::get_instance_count());
        
        res1 = new("Resource1");
        res2 = new("Resource2");
        
        $display("After creation: %0d", Resource::get_instance_count());
        
        // Objects are automatically garbage collected when no longer referenced
        res1 = null; // Remove reference
        
        // Explicit cleanup (if needed)
        res2.cleanup();
        res2 = null;
        
        // Note: instance_count doesn't decrease (no automatic destructor)
        $display("Final count: %0d", Resource::get_instance_count());
    end
endmodule
```

### **Example 13: Access Control Demo**
Class with public, protected, and local (private) members showing visibility rules.

In [17]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_13__access_control_demo/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// access_control_demo.sv
// Example demonstrating access control with public, protected, and local
// members in SystemVerilog classes

// Base class demonstrating different access levels
class base_account_class;
  // Public members - accessible from anywhere
  int public_account_id;
  string public_account_name;
  
  // Protected members - accessible within class and derived classes
  protected real protected_balance;
  protected bit protected_active_flag;
  
  // Local (private) members - only accessible within this class
  local string local_pin_code;
  local int local_security_level;
  
  function new(int id, string name, real balance, string pin);
    public_account_id = id;
    public_account_name = name;
    protected_balance = balance;
    protected_active_flag = 1'b1;
    local_pin_code = pin;
    local_security_level = 3;
    $display("Base account created: ID=%0d, Name=%s", id, name);
  endfunction
  
  // Public method to display account info
  function void display_public_info();
    $display("Public Info - ID: %0d, Name: %s", 
             public_account_id, public_account_name);
  endfunction
  
  // Protected method - accessible to derived classes
  protected function real get_balance();
    return protected_balance;
  endfunction
  
  // Local method - only accessible within this class
  local function bit verify_pin(string pin);
    return (pin == local_pin_code);
  endfunction
  
  // Public method that uses local method internally
  function bit authenticate(string pin);
    if (verify_pin(pin)) begin
      $display("Authentication successful for %s", public_account_name);
      return 1'b1;
    end else begin
      $display("Authentication failed for %s", public_account_name);
      return 1'b0;
    end
  endfunction
endclass

// Derived class inheriting from base class
class savings_account_class extends base_account_class;
  real interest_rate;
  
  function new(int id, string name, real balance, string pin, real rate);
    super.new(id, name, balance, pin);
    interest_rate = rate;
    $display("Savings account created with interest rate: %0.2f%%", 
             rate * 100);
  endfunction
  
  // Can access protected members from base class
  function void calculate_interest();
    real interest = protected_balance * interest_rate;
    protected_balance += interest;
    $display("Interest calculated: %0.2f, New balance: %0.2f", 
             interest, protected_balance);
  endfunction
  
  // Can access protected methods from base class
  function void show_balance();
    $display("Current balance: %0.2f", get_balance());
  endfunction
  
  // Cannot access local members from base class
  // This would cause a compilation error:
  // function void illegal_access();
  //   $display("PIN: %s", local_pin_code);  // ERROR!
  // endfunction
endclass
```

```systemverilog
// access_control_demo_testbench.sv
// Testbench demonstrating access control visibility rules

module access_control_testbench;
  
  base_account_class base_account;
  savings_account_class savings_account;
  
  initial begin
    // Dump waves
    $dumpfile("access_control_testbench.vcd");
    $dumpvars(0, access_control_testbench);
    
    $display("=== Access Control Demo ===");
    $display();
    
    // Create base account
    $display("--- Creating Base Account ---");
    base_account = new(1001, "John Doe", 1000.50, "1234");
    
    // Access public members - this works
    $display("Accessing public members:");
    $display("  Account ID: %0d", base_account.public_account_id);
    $display("  Account Name: %s", base_account.public_account_name);
    
    // Call public method
    base_account.display_public_info();
    
    // Test authentication (uses local method internally)
    $display("\n--- Testing Authentication ---");
    if (base_account.authenticate("1234")) begin  // Correct PIN
      $display("Access granted with correct PIN");
    end
    if (!base_account.authenticate("5678")) begin  // Wrong PIN
      $display("Access denied with incorrect PIN");
    end
    
    $display();
    
    // Create savings account (derived class)
    $display("--- Creating Savings Account ---");
    savings_account = new(2001, "Jane Smith", 2500.75, "9876", 0.05);
    
    // Access public members of derived class
    $display("Accessing public members of derived class:");
    $display("  Account ID: %0d", savings_account.public_account_id);
    $display("  Account Name: %s", savings_account.public_account_name);
    
    // Call methods that access protected members
    $display("\n--- Testing Protected Access ---");
    savings_account.show_balance();
    savings_account.calculate_interest();
    savings_account.show_balance();
    
    $display();
    
    // The following would cause compilation errors:
    // $display("Balance: %0.2f", base_account.protected_balance);     // ERROR!
    // $display("PIN: %s", base_account.local_pin_code);              // ERROR!
    // base_account.verify_pin("1234");                               // ERROR!
    
    $display("--- Access Control Rules Summary ---");
    $display("Public members: Accessible from anywhere");
    $display("Protected members: Accessible within class and derived classes");
    $display("Local members: Only accessible within the defining class");
    $display("Derived classes can access protected but not local members");
    
    $display();
    $display("=== Demo Complete ===");
    
    #10;
  end
  
endmodule
```

Verilator Simulation Output:
=== Access Control Demo ===

--- Creating Base Account ---
Base account created: ID=1001, Name=John Doe
Accessing public members:
  Account ID: 1001
  Account Name: John Doe
Public Info - ID: 1001, Name: John Doe

--- Testing Authentication ---
Authentication successful for John Doe
Access granted with correct PIN
Authentication failed for John Doe
Access denied with incorrect PIN

--- Creating Savings Account ---
Base account created: ID=2001, Name=Jane Smith
Savings account created with interest rate: 5.00%
Accessing public members of derived class:
  Account ID: 2001
  Account Name: Jane Smith

--- Testing Protected Access ---
Current balance: 2500.75
Interest calculated: 125.04, New balance: 2625.79
Current balance: 2625.79

--- Access Control Rules Summary ---
Public members: Accessible from anywhere
Protected members: Accessible within class and derived classes
Local members: Only accessible within the defining class
Derived classes can access protect

0

### **Example 14: Secure Data Class**
Demonstrates encapsulation with different access levels and methods to access private data.

In [21]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_14__secure_data_class/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// secure_data_class.sv
class secure_data_class;
  // Private data members (local by default in SystemVerilog)
  local int secret_code;
  local string confidential_message;
  
  // Protected data - accessible by derived classes
  int access_level;
  
  // Public data - accessible from anywhere (default)
  string public_info;
  
  // Constructor
  function new(int code = 1234, string msg = "Default Secret");
    this.secret_code = code;
    this.confidential_message = msg;
    this.access_level = 1;
    this.public_info = "Public Information";
  endfunction
  
  // Public method to set private data (setter)
  function void set_secret_code(int new_code);
    if (new_code > 0) begin
      this.secret_code = new_code;
      $display("Secret code updated successfully");
    end else begin
      $display("Invalid code! Must be positive");
    end
  endfunction
  
  // Public method to get private data (getter) - with validation
  function int get_secret_code(string password);
    if (password == "admin_pass") begin
      $display("Access granted - returning secret code");
      return this.secret_code;
    end else begin
      $display("Access denied - incorrect password");
      return -1;
    end
  endfunction
  
  // Public method to safely display information
  function void display_info();
    $display("=== Secure Data Class Information ===");
    $display("Public Info: %s", this.public_info);
    $display("Access Level: %0d", this.access_level);
    $display("Secret Code: [PROTECTED]");
    $display("Confidential Message: [PROTECTED]");
  endfunction
  
  // Method to verify access
  function bit verify_access(int code_attempt);
    return (code_attempt == this.secret_code);
  endfunction
  
endclass

module secure_data_module;
  initial begin
    $display("=== Secure Data Class Example ===");
    $display();
  end
endmodule
```

```systemverilog
// secure_data_class_testbench.sv
module secure_data_testbench;
  secure_data_module SECURE_DATA_INSTANCE();
  
  initial begin
    secure_data_class my_secure_data;
    int retrieved_code;
    bit access_result;
    
    // Dump waves
    $dumpfile("secure_data_testbench.vcd");
    $dumpvars(0, secure_data_testbench);
    
    $display("=== Testing Secure Data Class ===");
    $display();
    
    // Create instance of secure data class
    my_secure_data = new(9876, "Top Secret Information");
    
    // Test public access
    $display("1. Testing public access:");
    $display("   Public info: %s", my_secure_data.public_info);
    my_secure_data.public_info = "Updated Public Info";
    $display("   Updated public info: %s", my_secure_data.public_info);
    $display();
    
    // Test protected/public access
    $display("2. Testing public/protected access:");
    $display("   Access level: %0d", my_secure_data.access_level);
    $display();
    
    // Test private access through public methods
    $display("3. Testing local/private access through methods:");
    
    // Try to get secret code with wrong password
    $display("   Attempting access with wrong password:");
    retrieved_code = my_secure_data.get_secret_code("wrong_pass");
    $display("   Retrieved code: %0d", retrieved_code);
    $display();
    
    // Try to get secret code with correct password
    $display("   Attempting access with correct password:");
    retrieved_code = my_secure_data.get_secret_code("admin_pass");
    $display("   Retrieved code: %0d", retrieved_code);
    $display();
    
    // Test setter method
    $display("4. Testing setter method:");
    my_secure_data.set_secret_code(5555);
    $display();
    
    // Test access verification
    $display("5. Testing access verification:");
    access_result = my_secure_data.verify_access(1234);
    $display("   Verify with old code (1234): %s", 
             access_result ? "PASS" : "FAIL");
    
    access_result = my_secure_data.verify_access(5555);
    $display("   Verify with new code (5555): %s", 
             access_result ? "PASS" : "FAIL");
    $display();
    
    // Display secure information
    $display("6. Displaying secure information:");
    my_secure_data.display_info();
    $display();
    
    // Test invalid input
    $display("7. Testing invalid input:");
    my_secure_data.set_secret_code(-100);
    $display();
    
    $display("=== Secure Data Class Test Complete ===");
    
    #1;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Secure Data Class Example ===

=== Testing Secure Data Class ===

1. Testing public access:
   Public info: Public Information
   Updated public info: Updated Public Info

2. Testing public/protected access:
   Access level: 1

3. Testing local/private access through methods:
   Attempting access with wrong password:
Access denied - incorrect password
   Retrieved code: -1

   Attempting access with correct password:
Access granted - returning secret code
   Retrieved code: 9876

4. Testing setter method:
Secret code updated successfully

5. Testing access verification:
   Verify with old code (1234): FAIL
   Verify with new code (5555): PASS

6. Displaying secure information:
=== Secure Data Class Information ===
Public Info: Updated Public Info
Access Level: 1
Secret Code: [PROTECTED]
Confidential Message: [PROTECTED]

7. Testing invalid input:
Invalid code! Must be positive

=== Secure Data Class Test Complete ===
Process finished with return code: 0

0

### **Example 15: Resource Tracking**
Class that tracks object creation and cleanup with instance counting.

In [34]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_15__resource_tracking/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// resource_manager.sv
package resource_pkg;

  // Resource tracking class with instance counting
  class ResourceTracker;
    static int total_instances = 0;    // Total instances created
    static int active_instances = 0;   // Currently active instances
    int instance_id;                   // Unique ID for this instance
    string resource_name;              // Name of the resource
    
    // Constructor - tracks creation
    function new(string name = "unnamed_resource");
      total_instances++;
      active_instances++;
      instance_id = total_instances;
      resource_name = name;
      $display("Resource created: %s (ID: %0d)", resource_name, instance_id);
      $display("  Total created: %0d, Active: %0d", 
               total_instances, active_instances);
    endfunction
    
    // Destructor - tracks cleanup  
    function void cleanup();
      active_instances--;
      $display("Resource cleaned up: %s (ID: %0d)", 
               resource_name, instance_id);
      $display("  Total created: %0d, Active: %0d", 
               total_instances, active_instances);
    endfunction
    
    // Static method to get current statistics
    static function void print_stats();
      $display("Resource Statistics:");
      $display("  Total instances created: %0d", total_instances);
      $display("  Currently active: %0d", active_instances);
    endfunction
    
    // Method to display resource info
    function void display_info();
      $display("Resource Info: %s (ID: %0d)", resource_name, instance_id);
    endfunction
    
  endclass

endpackage

module resource_manager_module();

  // Simple demonstration - main logic is in testbench
  initial begin
    $display("Resource Manager Design Module Ready");
    $display();
  end

endmodule
```

```systemverilog
// resource_manager_testbench.sv
module resource_manager_testbench;

  // Import the resource tracking package
  import resource_pkg::*;

  // Import the design module
  resource_manager_module RESOURCE_MANAGER_INSTANCE();

  // Test task to demonstrate resource tracking
  task run_resource_test();
    // Create some resource instances using automatic variables
    automatic ResourceTracker memory_pool;
    automatic ResourceTracker file_handle;
    automatic ResourceTracker network_socket;
    automatic ResourceTracker database_conn;
    
    $display("Creating resources...");
    memory_pool = new("Memory Pool");
    file_handle = new("File Handle");
    network_socket = new("Network Socket");
    
    $display();
    ResourceTracker::print_stats();
    $display();
    
    // Display individual resource info
    $display("Individual resource information:");
    memory_pool.display_info();
    file_handle.display_info();
    network_socket.display_info();
    
    $display();
    $display("Cleaning up resources...");
    memory_pool.cleanup();
    file_handle.cleanup();
    
    $display();
    ResourceTracker::print_stats();
    
    $display();
    $display("Creating additional resource...");
    database_conn = new("Database Connection");
    
    $display();
    ResourceTracker::print_stats();
    
    $display();
    $display("Final cleanup...");
    network_socket.cleanup();
    database_conn.cleanup();
    
    $display();
    ResourceTracker::print_stats();
    
  endtask

  initial begin
    // Dump waves for verilator
    $dumpfile("resource_manager_testbench.vcd");
    $dumpvars(0, resource_manager_testbench);
    
    $display("=== Resource Tracking Example ===");
    $display();
    
    // Run the test
    run_resource_test();
    
    $display();
    $display("=== Resource Tracking Complete ===");
    
    #1;  // Wait for a time unit
  end

endmodule
```

Verilator Simulation Output:
Resource Manager Design Module Ready

=== Resource Tracking Example ===

Creating resources...
Resource created: Memory Pool (ID: 1)
  Total created: 1, Active: 1
Resource created: File Handle (ID: 2)
  Total created: 2, Active: 2
Resource created: Network Socket (ID: 3)
  Total created: 3, Active: 3

Resource Statistics:
  Total instances created: 3
  Currently active: 3

Individual resource information:
Resource Info: Memory Pool (ID: 1)
Resource Info: File Handle (ID: 2)
Resource Info: Network Socket (ID: 3)

Cleaning up resources...
Resource cleaned up: Memory Pool (ID: 1)
  Total created: 3, Active: 2
Resource cleaned up: File Handle (ID: 2)
  Total created: 3, Active: 1

Resource Statistics:
  Total instances created: 3
  Currently active: 1

Creating additional resource...
Resource created: Database Connection (ID: 4)
  Total created: 4, Active: 2

Resource Statistics:
  Total instances created: 4
  Currently active: 2

Final cleanup...
Resource clea

0

## Static Members

Static members belong to the class rather than to individual instances. They are shared among all objects of the class.

### Static Properties and Methods

```systemverilog
class IDGenerator;
    static int next_id = 1;
    static int total_objects = 0;
    static string class_version = "v1.0";
    
    int object_id;
    string name;
    
    function new(string name);
        this.name = name;
        this.object_id = next_id++;
        total_objects++;
        $display("Created object %0d: %s", object_id, name);
    endfunction
    
    // Static method - can be called without creating an object
    static function int get_next_id();
        return next_id;
    endfunction
    
    static function int get_total_objects();
        return total_objects;
    endfunction
    
    static function void reset_counter();
        next_id = 1;
        total_objects = 0;
        $display("ID counter reset");
    endfunction
    
    // Static method to get class information
    static function string get_class_info();
        return $sformatf("IDGenerator %s - Next ID: %0d, Total: %0d",
                        class_version, next_id, total_objects);
    endfunction
    
    // Instance method that uses static data
    function void show_info();
        $display("Object %0d (%s) - Class has %0d total objects",
                object_id, name, total_objects);
    endfunction
endclass

// Usage of static members
module test_static;
    IDGenerator obj1, obj2, obj3;
    
    initial begin
        // Call static method without creating objects
        $display("Class info: %s", IDGenerator::get_class_info());
        $display("Next ID will be: %0d", IDGenerator::get_next_id());
        
        // Create objects
        obj1 = new("First");
        obj2 = new("Second");
        obj3 = new("Third");
        
        // Show object info
        obj1.show_info();
        obj2.show_info();
        
        // Access static members through class name
        $display("Total objects created: %0d", IDGenerator::get_total_objects());
        
        // Reset static data
        IDGenerator::reset_counter();
        
        // Create new object after reset
        obj1 = new("After Reset");
        $display("Final class info: %s", IDGenerator::get_class_info());
    end
endmodule
```

### Static vs Instance Members

```systemverilog
class BankAccount;
    static real interest_rate = 0.05;  // Static - same for all accounts
    static int total_accounts = 0;     // Static - count of all accounts
    
    int account_number;                // Instance - unique per account
    real balance;                      // Instance - individual balance
    string owner_name;                 // Instance - individual owner
    
    function new(string name, real initial_balance = 0.0);
        total_accounts++;
        account_number = total_accounts;
        owner_name = name;
        balance = initial_balance;
    endfunction
    
    // Static method to change interest rate for all accounts
    static function void set_interest_rate(real new_rate);
        interest_rate = new_rate;
        $display("Interest rate changed to %.2f%% for all accounts", 
                 new_rate * 100);
    endfunction
    
    // Instance method that uses both static and instance data
    function void apply_interest();
        real interest = balance * interest_rate;
        balance += interest;
        $display("Account %0d (%s): Interest $%.2f applied, new balance $%.2f",
                account_number, owner_name, interest, balance);
    endfunction
    
    // Instance method
    function void deposit(real amount);
        balance += amount;
        $display("Account %0d: Deposited $%.2f, balance now $%.2f",
                account_number, amount, balance);
    endfunction
    
    static function void print_statistics();
        $display("Bank Statistics:");
        $display("  Total accounts: %0d", total_accounts);
        $display("  Current interest rate: %.2f%%", interest_rate * 100);
    endfunction
endclass

module test_static_vs_instance;
    BankAccount acc1, acc2, acc3;
    
    initial begin
        // Create accounts
        acc1 = new("Alice", 1000.0);
        acc2 = new("Bob", 500.0);
        acc3 = new("Charlie", 1500.0);
        
        // Show initial statistics
        BankAccount::print_statistics();
        
        // Apply interest with current rate
        acc1.apply_interest();
        acc2.apply_interest();
        acc3.apply_interest();
        
        // Change interest rate (affects all accounts)
        BankAccount::set_interest_rate(0.08);
        
        // Apply new interest rate
        acc1.apply_interest();
        acc2.apply_interest();
        acc3.apply_interest();
        
        // Final statistics
        BankAccount::print_statistics();
    end
endmodule
```

### **Example 16: ID Generator Class**
Class using static properties to generate unique IDs and track total object count.

In [2]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_16__id_generator_class/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// id_generator_class.sv
// ID Generator Class using static properties for unique IDs and object count

class id_generator_class;
  // Static properties - shared across all instances
  static int next_id = 1;           // Next ID to assign
  static int total_objects = 0;     // Total objects created
  
  // Instance properties
  int object_id;                    // Unique ID for this instance
  string object_name;               // Name of the object
  
  // Constructor
  function new(string name = "unnamed");
    object_id = next_id++;          // Assign unique ID and increment
    object_name = name;             // Set object name
    total_objects++;                // Increment total object count
    
    $display("Created object: %s with ID: %0d", object_name, object_id);
  endfunction
  
  // Static function to get total object count
  static function int get_total_objects();
    return total_objects;
  endfunction
  
  // Static function to get next ID that will be assigned
  static function int get_next_id();
    return next_id;
  endfunction
  
  // Instance method to display object info
  function void display_info();
    $display("Object Info - Name: %s, ID: %0d", object_name, object_id);
  endfunction
  
  // Static function to display class statistics
  static function void display_statistics();
    $display("=== ID Generator Statistics ===");
    $display("Total objects created: %0d", total_objects);
    $display("Next ID to assign: %0d", next_id);
    $display("===============================");
  endfunction
  
endclass
```

```systemverilog
// id_generator_class_testbench.sv
// Testbench for ID Generator Class demonstrating static properties

module id_generator_testbench;
  
  // Declare class handles
  id_generator_class obj1, obj2, obj3, obj4;
  
  initial begin
    // Dump waves for Verilator
    $dumpfile("id_generator_testbench.vcd");
    $dumpvars(0, id_generator_testbench);
    
    $display("=== ID Generator Class Example ===");
    $display();
    
    // Display initial statistics (should be 0 objects)
    $display("Initial state:");
    id_generator_class::display_statistics();
    $display();
    
    // Create first object
    $display("Creating first object (CPU)...");
    obj1 = new("CPU");
    obj1.display_info();
    $display("Objects created so far: %0d", 
             id_generator_class::get_total_objects());
    $display();
    
    // Create second object
    $display("Creating second object (Memory)...");
    obj2 = new("Memory");
    obj2.display_info();
    $display("Objects created so far: %0d", 
             id_generator_class::get_total_objects());
    $display();
    
    // Create third object
    $display("Creating third object (Cache)...");
    obj3 = new("Cache");
    obj3.display_info();
    $display("Next ID will be: %0d", id_generator_class::get_next_id());
    $display();
    
    // Display all object information
    $display("=== All Object Information ===");
    obj1.display_info();
    obj2.display_info();
    obj3.display_info();
    $display();
    
    // Create fourth object with default name
    $display("Creating fourth object (default name)...");
    obj4 = new();
    obj4.display_info();
    $display();
    
    // Display final statistics
    $display("Final statistics:");
    id_generator_class::display_statistics();
    $display();
    
    // Demonstrate that static properties are shared
    $display("=== Demonstrating Static Property Sharing ===");
    $display("Accessing total count from obj1: %0d", 
             obj1.get_total_objects());
    $display("Accessing total count from obj2: %0d", 
             obj2.get_total_objects());
    $display("Accessing total count from class: %0d", 
             id_generator_class::get_total_objects());
    $display("All should be the same value!");
    $display();
    
    $display("=== Example Complete ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

Initial state:
=== ID Generator Statistics ===
Total objects created: 0
Next ID to assign: 1

Creating first object (CPU)...
Created object: CPU with ID: 1
Object Info - Name: CPU, ID: 1
Objects created so far: 1

Creating second object (Memory)...
Created object: Memory with ID: 2
Object Info - Name: Memory, ID: 2
Objects created so far: 2

Creating third object (Cache)...
Created object: Cache with ID: 3
Object Info - Name: Cache, ID: 3
Next ID will be: 4

=== All Object Information ===
Object Info - Name: CPU, ID: 1
Object Info - Name: Memory, ID: 2
Object Info - Name: Cache, ID: 3

Creating fourth object (default name)...
Created object: unnamed with ID: 4
Object Info - Name: unnamed, ID: 4

Final statistics:
=== ID Generator Statistics ===
Total objects created: 4
Next ID to assign: 5

=== Demonstrating Static Property Sharing ===
Accessing total count from obj1: 4
Accessing total count from obj2: 4
Accessing total count from class: 4
All should be th

0

### **Example 17: Static Utility Methods**
Examples of static methods that can be called without creating class instances.

In [5]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_17__static_utility_methods/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// math_utilities.sv
// Demonstrates static utility methods that can be called without instances

class MathUtilities;
  
  // Static method to find maximum of two numbers
  static function int max(int a, int b);
    return (a > b) ? a : b;
  endfunction
  
  // Static method to find minimum of two numbers
  static function int min(int a, int b);
    return (a < b) ? a : b;
  endfunction
  
  // Static method to calculate absolute value
  static function int abs(int value);
    return (value < 0) ? -value : value;
  endfunction
  
  // Static method to check if number is even
  static function bit is_even(int number);
    return (number % 2 == 0);
  endfunction
  
  // Static method to calculate power of 2
  static function int power_of_two(int exponent);
    return (1 << exponent);
  endfunction
  
  // Static method to clamp value between min and max
  static function int clamp(int value, int min_val, int max_val);
    if (value < min_val) return min_val;
    if (value > max_val) return max_val;
    return value;
  endfunction
  
endclass

// String utilities class
class StringUtilities;
  
  // Static method to get string length (simplified)
  static function int str_length(string str);
    return str.len();
  endfunction
  
  // Static method to check if string is empty
  static function bit is_empty(string str);
    return (str.len() == 0);
  endfunction
  
  // Static method to convert to uppercase (first character only)
  static function string to_upper_first(string str);
    byte ascii_val;
    string result;
    
    if (str.len() == 0) return "";
    
    ascii_val = str.getc(0);
    if (ascii_val >= 97 && ascii_val <= 122) begin  // lowercase a-z
      ascii_val = ascii_val - 32;  // Convert to uppercase
      result = {string'(ascii_val), str.substr(1, str.len()-1)};
    end else begin
      result = str;
    end
    
    return result;
  endfunction
  
endclass
```

```systemverilog
// math_utilities_testbench.sv
// Testbench demonstrating static utility methods

module math_utilities_testbench;
  
  initial begin
    // Setup VCD dumping
    $dumpfile("math_utilities_testbench.vcd");
    $dumpvars(0, math_utilities_testbench);
    
    $display("=== Static Utility Methods Demo ===");
    $display();
    
    // Test MathUtilities static methods (no instance needed!)
    $display("--- Math Utilities ---");
    $display("Max(15, 8) = %0d", MathUtilities::max(15, 8));
    $display("Min(15, 8) = %0d", MathUtilities::min(15, 8));
    $display("Abs(-42) = %0d", MathUtilities::abs(-42));
    $display("Abs(42) = %0d", MathUtilities::abs(42));
    $display("Is 10 even? %0s", MathUtilities::is_even(10) ? "Yes" : "No");
    $display("Is 7 even? %0s", MathUtilities::is_even(7) ? "Yes" : "No");
    $display("2^5 = %0d", MathUtilities::power_of_two(5));
    $display("2^3 = %0d", MathUtilities::power_of_two(3));
    $display("Clamp(25, 10, 20) = %0d", 
             MathUtilities::clamp(25, 10, 20));
    $display("Clamp(5, 10, 20) = %0d", 
             MathUtilities::clamp(5, 10, 20));
    $display("Clamp(15, 10, 20) = %0d", 
             MathUtilities::clamp(15, 10, 20));
    
    $display();
    
    // Test StringUtilities static methods
    $display("--- String Utilities ---");
    $display("Length of 'Hello' = %0d", 
             StringUtilities::str_length("Hello"));
    $display("Is '' empty? %0s", 
             StringUtilities::is_empty("") ? "Yes" : "No");
    $display("Is 'test' empty? %0s", 
             StringUtilities::is_empty("test") ? "Yes" : "No");
    $display("Upper first 'hello' = '%0s'", 
             StringUtilities::to_upper_first("hello"));
    $display("Upper first 'WORLD' = '%0s'", 
             StringUtilities::to_upper_first("WORLD"));
    $display("Upper first 'systemVerilog' = '%0s'", 
             StringUtilities::to_upper_first("systemVerilog"));
    
    $display();
    
    // Demonstrate that we don't need class instances
    $display("--- Key Point: No Class Instances Required ---");
    $display("All methods called using ClassName::method_name()");
    $display("This is the power of static methods!");
    
    $display();
    $display("=== Test Complete ===");
    
    #10;  // Wait a bit
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Static Utility Methods Demo ===

--- Math Utilities ---
Max(15, 8) = 15
Min(15, 8) = 8
Abs(-42) = 42
Abs(42) = 42
Is 10 even? Yes
Is 7 even? No
2^5 = 32
2^3 = 8
Clamp(25, 10, 20) = 20
Clamp(5, 10, 20) = 10
Clamp(15, 10, 20) = 15

--- String Utilities ---
Length of 'Hello' = 5
Is '' empty? Yes
Is 'test' empty? No
Upper first 'hello' = 'Hello'
Upper first 'WORLD' = 'WORLD'
Upper first 'systemVerilog' = 'SystemVerilog'

--- Key Point: No Class Instances Required ---
All methods called using ClassName::method_name()
This is the power of static methods!

=== Test Complete ===
Process finished with return code: 0
Removing Chapter_9_examples/example_17__static_utility_methods/obj_dir directory...
Chapter_9_examples/example_17__static_utility_methods/obj_dir removed successfully.


0

### **Example 18: Bank Account System**
Demonstrates the difference between static (shared) and instance (individual) data members.

In [9]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_9_examples/example_18__bank_account_system/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bank_account_system.sv
package bank_pkg;
  
  class BankAccount;
    // Static data members - shared across all instances
    static int total_accounts = 0;           // Total number of accounts
    static real total_bank_balance = 0.0;    // Sum of all account balances
    static string bank_name = "SystemVerilog Bank";
    
    // Instance data members - unique to each account
    int account_number;                      // Individual account number
    string account_holder;                   // Account holder name
    real balance;                           // Individual account balance
    
    // Constructor
    function new(string holder_name, real initial_balance = 0.0);
      total_accounts++;                      // Increment static counter
      account_number = total_accounts;       // Assign unique account number
      account_holder = holder_name;
      balance = initial_balance;
      total_bank_balance += initial_balance; // Add to total bank balance
      
      $display("Account created: #%0d for %s with balance $%.2f", 
               account_number, account_holder, balance);
    endfunction
    
    // Method to deposit money
    function void deposit(real amount);
      if (amount > 0) begin
        balance += amount;
        total_bank_balance += amount;        // Update static total
        $display("Deposited $%.2f to account #%0d. New balance: $%.2f", 
                 amount, account_number, balance);
      end else begin
        $display("Invalid deposit amount: $%.2f", amount);
      end
    endfunction
    
    // Method to withdraw money
    function void withdraw(real amount);
      if (amount > 0 && amount <= balance) begin
        balance -= amount;
        total_bank_balance -= amount;        // Update static total
        $display("Withdrew $%.2f from account #%0d. New balance: $%.2f", 
                 amount, account_number, balance);
      end else begin
        $display("Invalid withdrawal: $%.2f (available: $%.2f)", 
                 amount, balance);
      end
    endfunction
    
    // Method to display account info
    function void display_account();
      $display("Account #%0d: %s - Balance: $%.2f", 
               account_number, account_holder, balance);
    endfunction
    
    // Static method to display bank statistics
    static function void display_bank_stats();
      $display("=== %s Statistics ===", bank_name);
      $display("Total Accounts: %0d", total_accounts);
      $display("Total Bank Balance: $%.2f", total_bank_balance);
      $display("Average Balance: $%.2f", 
               total_accounts > 0 ? total_bank_balance / total_accounts : 0.0);
    endfunction
    
  endclass
  
endpackage

module bank_account_system;
  // This module serves as a container for the BankAccount class
  // The actual functionality is demonstrated in the testbench
endmodule
```

```systemverilog
// bank_account_system_testbench.sv
// No include needed - compile both files together

module bank_account_testbench;
  
  // Import the package
  import bank_pkg::*;
  
  // Declare account handles
  BankAccount alice_account, bob_account, charlie_account;
  
  initial begin
    // Dump waves for Verilator
    $dumpfile("bank_account_testbench.vcd");
    $dumpvars(0, bank_account_testbench);
    
    $display("=== Bank Account System Demo ===");
    $display("Demonstrating static vs instance data members");
    $display();
    
    // Show initial bank statistics (before any accounts)
    $display("Initial bank state:");
    BankAccount::display_bank_stats();
    $display();
    
    // Create first account
    $display("Creating accounts...");
    alice_account = new("Alice Johnson", 1000.0);
    
    // Show how static members are updated
    $display("After creating Alice's account:");
    BankAccount::display_bank_stats();
    $display();
    
    // Create second account
    bob_account = new("Bob Smith", 500.0);
    
    // Create third account
    charlie_account = new("Charlie Brown", 750.0);
    $display();
    
    // Show updated bank statistics
    $display("After creating all accounts:");
    BankAccount::display_bank_stats();
    $display();
    
    // Display individual account information
    $display("Individual account details:");
    alice_account.display_account();
    bob_account.display_account();
    charlie_account.display_account();
    $display();
    
    // Perform some transactions
    $display("Performing transactions...");
    alice_account.deposit(200.0);
    bob_account.withdraw(100.0);
    charlie_account.deposit(50.0);
    $display();
    
    // Show how static total_bank_balance is updated
    $display("After transactions:");
    BankAccount::display_bank_stats();
    $display();
    
    // Show individual balances (instance members)
    $display("Updated individual account details:");
    alice_account.display_account();
    bob_account.display_account();
    charlie_account.display_account();
    $display();
    
    // Demonstrate that each account has its own instance data
    $display("=== Key Observations ===");
    $display("1. Static members (total_accounts, total_bank_balance):");
    $display("   - Shared across ALL account instances");
    $display("   - Updated when ANY account is modified");
    $display("   - Can be accessed via class name: BankAccount::total_accounts");
    $display();
    $display("2. Instance members (account_number, balance, account_holder):");
    $display("   - Unique to each account object");
    $display("   - Alice's balance: $%.2f", alice_account.balance);
    $display("   - Bob's balance: $%.2f", bob_account.balance);
    $display("   - Charlie's balance: $%.2f", charlie_account.balance);
    $display();
    
    // Final bank statistics
    $display("Final bank statistics:");
    BankAccount::display_bank_stats();
    
    #10;  // Wait before finishing
    $display("Simulation completed!");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Bank Account System Demo ===
Demonstrating static vs instance data members

Initial bank state:
=== SystemVerilog Bank Statistics ===
Total Accounts: 0
Total Bank Balance: $0.00
Average Balance: $0.00

Creating accounts...
Account created: #1 for Alice Johnson with balance $1000.00
After creating Alice's account:
=== SystemVerilog Bank Statistics ===
Total Accounts: 1
Total Bank Balance: $1000.00
Average Balance: $1000.00

Account created: #2 for Bob Smith with balance $500.00
Account created: #3 for Charlie Brown with balance $750.00

After creating all accounts:
=== SystemVerilog Bank Statistics ===
Total Accounts: 3
Total Bank Balance: $2250.00
Average Balance: $750.00

Individual account details:
Account #1: Alice Johnson - Balance: $1000.00
Account #2: Bob Smith - Balance: $500.00
Account #3: Charlie Brown - Balance: $750.00

Performing transactions...
Deposited $200.00 to account #1. New balance: $1200.00
Withdrew $100.00 from account #2. New bala

0

### Best Practices

1. **Use constructors** to initialize object state properly
2. **Implement deep copy methods** when objects contain dynamic arrays or other objects
3. **Use `this` keyword** to resolve naming conflicts and improve code clarity
4. **Apply appropriate access control** (public, protected, local) to encapsulate data
5. **Use static members** for class-wide data and utility functions
6. **Implement cleanup methods** for resources that need explicit cleanup
7. **Design classes with single responsibility** for better maintainability

### Summary

Classes and objects in SystemVerilog provide powerful abstraction mechanisms for creating reusable and maintainable code. Key concepts include:

- **Class declarations** define templates for objects with properties and methods
- **Object creation** uses constructors and the `new()` operator
- **The `this` keyword** provides explicit reference to the current object
- **Class scope** controls member visibility and access
- **Static members** are shared across all instances of a class
- **Proper encapsulation** and access control improve code reliability

Understanding these concepts is essential for effective object-oriented programming in SystemVerilog, particularly for complex verification environments and testbenches.

# SystemVerilog Classes and Objects - Simple Examples

Based on Chapter 9: Classes and Objects, here are super simple examples organized by subchapter:

## Best Practices Examples

### **Constructor Best Practices**
Proper initialization of object state with parameter validation and default values.

### **Deep Copy Implementation**
How to implement proper deep copy methods for objects containing dynamic arrays.

### **Encapsulation Example**
Well-designed class showing proper use of access control and getter/setter methods.

### **Resource Management**
Class design patterns for objects that need explicit cleanup or resource management.

### **Single Responsibility**
Example of a class designed with a single, well-defined purpose for better maintainability.

### **Static vs Instance Usage**
Clear examples showing when to use static members vs instance members appropriately.